<a href="https://colab.research.google.com/github/waytoakash/waytoakash/blob/main/Sales_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
from datetime import timedelta
import re

from dateutil.relativedelta import relativedelta #
from functools import reduce

from google.colab import drive
drive.mount('/content/drive', force_remount = True)

import gspread
from google.colab import auth
auth.authenticate_user() #

from oauth2client.client import GoogleCredentials #
#gc = gspread.authorize(GoogleCredentials.get_application_default()) #

auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

from gspread_dataframe import set_with_dataframe

!pip install fuzzywuzzy
!pip install python-Levenshtein

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

def changeDateFormat(row):
  return pd.to_datetime(row, errors='coerce').dt.date

def changeDateTimeFormat(row):
  return pd.to_datetime(row, errors='coerce')

def cleanText(row):
  return row.str.upper().str.strip()

def getDump(mastermerge, dump_name,start_date, end_date):
  start_date = pd.to_datetime(start_date).date()
  end_date = pd.to_datetime(end_date).date()
  file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/"
  reg = re.compile(".+" + dump_name+".csv")

  for (root, dirs, files) in os.walk(file_path):    
      for file in files:
        path = os.path.join(root, file)
        date = str(path).split('/')[7:][0]
        date = pd.to_datetime(date).date()        
        
        if bool(re.match(reg, path)) & (date >= start_date) & (date <= end_date):                 
          temp = pd.read_csv(path,parse_dates=True)        
          temp['DumpDate'] = date
          read_dump_name = str(path).split('/')[9:][0]
          #print(read_dump_name + " on Date:: " + str(date) +  " Dump of shape:: " + str(temp.shape) + " read from path:: "+ path)
          mastermerge = mastermerge.append(temp)

  mastermerge[['DumpDate']] = mastermerge[['DumpDate']].apply(changeDateFormat)
  return mastermerge

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
daysDelta =0

DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')

Last30DaysDate = (datetime.today() - timedelta(days = 30 + daysDelta)).date()
Last60DaysDate = (datetime.today() - timedelta(days = 60 + daysDelta)).date()
Last90DaysDate = (datetime.today() - timedelta(days = 90 + daysDelta)).date()
Last7DaysDate = (datetime.today() - timedelta(days = 7 + daysDelta)).date()
Last45DaysDate = (datetime.today() - timedelta(days = 45 + daysDelta)).date()
MonthStartDate = datetime.today().date().replace(day=1)

start_date = Last30DaysDate.strftime('%d %B %Y')

file_path = "/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Aug 2022/"

#HR sheet

In [ ]:
def get_data(spreadSheetId, rangeName):

    wb = gc.open_by_key(spreadSheetId)
    ws = wb.worksheet(rangeName)
    rows = ws.get_all_values()
    return rows
#get all data in a sheet as df and there 1st row things as the df labels.
#If sheets don't have their labels in the first row than we have to do it manuaaly as per the data
def get_data_df(spreadSheetId, rangeName):
  l = get_data(spreadSheetId, rangeName)
  ldf = pd.DataFrame.from_records(l[1:], columns = l[0])
  return ldf

spreadSheetId="1IHV2Kge3YC7xqj1Gg0YdX0swzZHfJsxcSQuLf4CBGjE"
rangeName="Hood Headcount"
HR=get_data_df(spreadSheetId, rangeName)


HR = HR[['Emp Code', 'Official Email ID','Employee Name','Location', 'Department', 'Designation','Role','Reporting Supervisor E-mail ID',  
         'Status', 'DOJ', 'Reporting Date','LWD', 'On Field Status', 'In PIP/ On Notice Status','Source']]

HR.rename({'Official Email ID':'Emp Email ID','Location':'Emp City', 'Reporting Supervisor E-mail ID': 'Manager Email ID',  
         'Status':'Emp Status', 'Reporting Date':'DOR'}, inplace = True, axis = 1)
HR = HR[~(HR['Emp Code'].isna())]
HR = HR[HR['Emp Code'] != '']
HR[['DOJ','DOR','LWD']] = HR[['DOJ','DOR','LWD']].apply(changeDateFormat)
HR[['Emp Code','Role']] = HR[['Emp Code','Role']].apply(cleanText)
HR['Emp Email ID'] = HR['Emp Email ID'].str.strip().str.lower()
HR.loc[HR['Emp City'] == 'Bengaluru','Emp City'] = 'Bangalore'

HR_Sales = HR[HR['Department'] == "Sales"]

#Read RAW data

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/16j5O0DC1SIWqElPjiNWvww4UmwqqNTzMtic6gK3UdIA/edit?resourcekey#gid=1285863584')
sheet =  sheet.get_worksheet(0)
closuresRaw = pd.DataFrame(sheet.get_all_records())

closuresRaw.rename({'Timestamp':'Closure Form Filled Date', 'Email Address':'Closure Form Filled By', 'Please select the Closure type':'Closure Type',
       'Uniview Closure Number (Current)':'Closure Number', 'City':'Closure City',
       'Specify the HANDOVER MEETING ID':'HO Meeting ID', 'Society Name':'Society Name (Sales)',
       'Closure date':'Closure Date', 'BD Signed - Primary Email':'Closure Done By (Primary Email)',
       'BD Signed - Secondary Email (if applicable)':'Closure Done By (Secondary Email)',
       'Society Type (Plotted colonies not allowed from March 2021)':'Society Type',
       'Product(s) Sold':'Sales Product', 'Please fill the ASP Name':'ASP Name',
       'Please upload relevant closure proof document(s) ':'Closure Proof Link',
       'Are we are replacing a competitor with this closure?':'Competition Replaced',
       'Please upload relevant document as proof for replacing VMS competitor':'Competition Proof Link (VMS)',
       'Please upload relevant document as a proof for replacing ERP competitor':'Competition Proof Link (ERP)',
       'I (ASM) have ensured that my TSM or me have filled the following forms [Sales Welfare commitment form ( Competitor ) - https://bit.ly/3MXGlwe]':'Liked Features Form Filled',       
       'Upload relevant proof if Premium Factor applicable':'Premimum Factor Proof Link',
       'If Other City Closure (NBH Non-Operational Cities), Email ID of KAM/Sr KAM/Sr KAM onboarding/FSD':'Ops Team Email ID (Other City Closure)',
       'If Other City Closure (NBH Non-Operational Cities), Base City of KAM/Sr KAM/Sr KAM onboarding/FSD':'Ops Team Base City (Other City Closure)'}, inplace = True, axis = 1)


def mergeCol(row):

  TotalDevicesVMS = ""
  TotalBiometricsVMS = ""
  VMSTrialPeriod = ""
  ERPTrialPeriod = ""
  MigrationDataReceived = ""
  MigrationMCConfirmation = ""

  if row['Sales Product'].strip() == "VMS only":

    TotalDevicesVMS = row['Total Devices (VMS Only)']
    TotalBiometricsVMS = row['Total Biometrics (VMS Only)']
    VMSTrialPeriod = row['VMS Trial Period (VMS Only)']
    
  elif row['Sales Product'].strip() == "ERP only":

    ERPTrialPeriod = row['ERP Trial Period (ERP Only)']
    MigrationDataReceived = row['Migration Data Received (ERP Only)']
    MigrationMCConfirmation = row['Does Client and sales confirm that they want billing data migration? (ERP Only)']

  elif row['Sales Product'].strip() == "Both VMS + ERP":

    TotalDevicesVMS = row['Total Devices (VMS+ERP)']
    TotalBiometricsVMS = row['Total Biometrics (VMS+ERP)']
    VMSTrialPeriod = row['VMS Trial Period (VMS+ERP)']
    ERPTrialPeriod = row['ERP Trial Period (VMS+ERP)']
    MigrationDataReceived = row['Migration Data Received (VMS+ERP)']
    MigrationMCConfirmation = row['Does Client and sales confirm that they want billing data migration? (VMS+ERP)']
    
  return TotalDevicesVMS,TotalBiometricsVMS,VMSTrialPeriod,ERPTrialPeriod,MigrationDataReceived,MigrationMCConfirmation

closuresRaw[['Total Devices (VMS)','Total Biometrics (VMS)','VMSTrialPeriod','ERPTrialPeriod','MigrationDataReceived','MigrationMCConfirmation']] = closuresRaw.apply(mergeCol, axis=1, result_type ='expand')


closuresRaw[['Closure Form Filled Time']] = closuresRaw[['Closure Form Filled Date']].apply(changeDateTimeFormat)
closuresRaw[['Closure Date','Closure Form Filled Date']] = closuresRaw[['Closure Date','Closure Form Filled Date']].apply(changeDateFormat)
closuresRaw = closuresRaw[(closuresRaw['Closure Date'] >= pd.to_datetime("2021-08-01").date())]

closuresRaw[['Society Name (Sales)']] = closuresRaw[['Society Name (Sales)']].apply(cleanText)
closuresRaw.loc[closuresRaw['Closure Type'] == 'New Closure', 'Original Closure Number'] = closuresRaw['Closure Number']

closuresRaw = closuresRaw[['Closure Type','Closure Date','Original Closure Number','Closure Number',
                     'Closure Form Filled Date', 'Closure Form Filled Time', 'Closure Form Filled By', 'Team Type', 'Lead Type', 'Closure Done By (Primary Email)','Closure Done By (Secondary Email)',
                     'Society Name (Sales)','Society Type', 'Property Type', 'Closure City', 'Locality','Society Latitude', 'Society Longitude', 
                     'Sales Product','Total Devices (VMS)', 'Total Biometrics (VMS)','VMSTrialPeriod','ERPTrialPeriod',
                     'Total Flats', 'Actual Flats','Sold Flats','VMS Sold Flats',
                     'MigrationDataReceived','MigrationMCConfirmation',
                     'Closure Proof Link','Premimum Factor Proof Link',
                     'Competition Replaced','Competition Proof Link (VMS)','Competition Proof Link (ERP)', 'VMS Competitor being replaced', 'ERP Competitor being replaced',
                     'Remarks on MC (if any)','Liked Features Form Filled','HO Meeting ID','ASP Name','Ops Team Email ID (Other City Closure)','Ops Team Base City (Other City Closure)']]

closuresRaw['Closure Done By (Primary Email)'] = closuresRaw['Closure Done By (Primary Email)'].str.strip().str.lower()
closuresRaw['Closure Done By (Secondary Email)'] = closuresRaw['Closure Done By (Secondary Email)'].str.strip().str.lower()
closuresRaw['Sold Flats'] = pd.to_numeric(closuresRaw['Sold Flats'], errors = 'coerce')
closuresRaw['VMS Sold Flats'] = pd.to_numeric(closuresRaw['VMS Sold Flats'], errors = 'coerce')
closuresRaw.loc[ (closuresRaw['Sales Product'].isin({'VMS only','Both VMS + ERP'}) & (closuresRaw['VMS Sold Flats'].isna())), 'VMS Sold Flats'] = closuresRaw['Sold Flats']

###########################
#Alerts

closuresRaw = pd.merge(closuresRaw, HR_Sales[['Emp Email ID','Emp City','Role','Designation','Manager Email ID']], left_on = 'Closure Done By (Primary Email)', right_on = 'Emp Email ID', how = 'left')
closuresRaw.rename({'Emp City':'Primary BD City','Role':'Primary BD Role', 'Designation':'Primary BD Designation','Manager Email ID':'Primary RM Mail'}, inplace = True, axis = 1)
closuresRaw.drop({'Emp Email ID'}, inplace = True, axis = 1)

closuresRaw.loc[closuresRaw['Primary BD Role'].isna(),'WrongPrimaryBDMail'] = True

closuresRaw = pd.merge(closuresRaw, HR_Sales[['Emp Email ID','Emp City','Role','Designation','Manager Email ID']], left_on = 'Closure Done By (Secondary Email)', right_on = 'Emp Email ID', how = 'left')
closuresRaw.rename({'Emp City':'Secondary BD City','Role':'Secondary BD Role', 'Designation':'Secondary BD Designation','Manager Email ID':'Secondary RM Mail'}, inplace = True, axis = 1)
closuresRaw.drop({'Emp Email ID'}, inplace = True, axis = 1)

closuresRaw.loc[(closuresRaw['Secondary BD Role'].isna()) & (closuresRaw['Closure Done By (Secondary Email)'] != ""),'WrongSecondaryBDMail'] = True

###########################

#closuresRaw = closuresRaw.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number','Closure Number']).head(5)

closuresRawDuplicateClosureNumber = closuresRaw[closuresRaw['Closure Number'].duplicated()]['Closure Number'].drop_duplicates()
closuresRaw.loc[closuresRaw['Closure Number'].isin(closuresRawDuplicateClosureNumber),'DuplicateClosureNumber'] = True

###########################

closuresRaw.loc[:,'Closure Form Filling TAT'] = (closuresRaw['Closure Form Filled Date'] - closuresRaw['Closure Date']).dt.days

###########################

closuresRaw.loc[:,'CrossCityClosure'] = (closuresRaw['Closure City'] != closuresRaw['Primary BD City']) | ( (~closuresRaw['Secondary BD City'].isna()) & (closuresRaw['Closure City'] != closuresRaw['Secondary BD City']))
closuresRaw.loc[closuresRaw['CrossCityClosure'] == False,'CrossCityClosure'] = ""
###########################

closuresRaw.loc[(closuresRaw['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})) & (closuresRaw['Closure Number'] == closuresRaw['Original Closure Number']),'WrongOriginalClosureNumber'] = True

###########################

closuresRawActivity = closuresRaw[closuresRaw['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})]

closuresRawActivity = closuresRaw.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number']).head(2)
closuresRawActivity = closuresRawActivity[(closuresRawActivity['Original Closure Number'].duplicated(keep= False)) & (closuresRawActivity['Original Closure Number']!= "")]

closuresRawActivity.loc[:, 'Closure Month'] = pd.to_datetime(closuresRawActivity['Closure Date']).dt.month
closuresRawActivity['Months Since Last Closure'] = closuresRawActivity.groupby(['Original Closure Number'])['Closure Month'].transform(lambda x: x.diff()) 
closuresRawActivity['Days Since Last Closure'] = closuresRawActivity.groupby(['Original Closure Number'])['Closure Date'].transform(lambda x: x.diff().dt.days) 

closuresRawActivity.sort_index(inplace=True)

closuresRawActivity.loc[closuresRawActivity['Days Since Last Closure']<=90, 'ClosureActivity'] = 'Within 90 days'
closuresRawActivity.loc[closuresRawActivity['Days Since Last Closure']<=35, 'ClosureActivity'] = 'Within 35 days'
closuresRawActivity.loc[closuresRawActivity['Months Since Last Closure']==0, 'ClosureActivity'] = 'Within Month Closure Activity'

closuresRawActivity = closuresRawActivity[~closuresRawActivity['ClosureActivity'].isna()][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates()
closuresRaw = pd.merge(closuresRaw,closuresRawActivity, on = ['Original Closure Number','Closure Number'], how = 'left')

###########################

# socnames = closuresRaw[['Society Name (Sales)']].drop_duplicates()
# for (idx, val) in enumerate(closuresRaw['Society Name (Sales)']):   
 
#     if ~pd.isnull(val):                   
#         MatchLimit = 3
#         ratio = process.extract( val , socnames[socnames.index != idx]['Society Name (Sales)'] , limit = MatchLimit , scorer = fuzz.ratio)

#         matchSoc = ""
#         matchSimilarity = ""
#         for i in range(1, MatchLimit):
#           if (ratio[i][1] >= 90):
#             matchSoc = matchSoc + ratio[i][0] + ","
#             matchSimilarity = matchSimilarity + str(ratio[i][1]) + ","        
        
#         closuresRaw.loc[idx, 'Possible SocietyName Matches'] = matchSoc
#         closuresRaw.loc[idx, 'similarity'] = matchSimilarity

# closuresRaw.loc[:, 'Possible SocietyName Matches'] = ""
# closuresRaw.loc[:, 'similarity'] = ""

###########################

#closuresRaw.loc[(closuresRaw['Sold Flats'].isna()) | (closuresRaw['Sold Flats'].str.replace(" ","") == ""), 'Sold Flats' ] = closuresRaw['Actual Flats']


sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1p9VJ0ATPxz68WMfR4uY0ZFb7YDN3Q51DVG69L2z88Go/edit?resourcekey#gid=1935498822')

sheet =  sheet.get_worksheet(0)
HORaw = pd.DataFrame(sheet.get_all_records())

HORaw.loc[(HORaw['VMS Relaxed HO?'] == "Yes") & (HORaw['VMS Handover Status (Relaxed HO)'] == 'Positive'), 'VMS Handover Status'] = 'Positive Relaxed'

HORaw = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date','Actual Society Name', 'Closure Product', 'Closure Type', 'Original Closure Number',      
       'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status',
       'ERP Uniview Closure Number', 'ERP Handover Status']]

HORaw[['Timestamp']] = HORaw[['Timestamp']].apply(changeDateTimeFormat)
HORaw[['Handover meeting date']] = HORaw[['Handover meeting date']].apply(changeDateFormat)
HORaw['VMS Uniview Closure Number'] = pd.to_numeric(HORaw['VMS Uniview Closure Number'])
HORaw['ERP Uniview Closure Number'] = pd.to_numeric(HORaw['ERP Uniview Closure Number'])


HOVMS = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status']]

HOVMS = HOVMS[~(HOVMS['VMS Uniview Closure Number'].isna())]
HOVMS = HOVMS.sort_values(['Handover meeting date'], ascending = True).groupby(['VMS Uniview Closure Number']).head(1)

HOVMS.rename({'VMS Uniview Closure Number':'Closure Number','VMS Handover Status':'VMSHOStatus','Handover meeting date':'VMSHODate',
              'Total Sold flats ':'Sold Flats (VMS)', 'Total Occupied Flats' :'Occupied Flats (VMS)'},inplace=True,axis=1)
HOVMS['Sold Flats (VMS)'] = pd.to_numeric(HOVMS['Sold Flats (VMS)'], errors = 'coerce')

HOERP = HORaw[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'ERP Uniview Closure Number','ERP Handover Status']]
HOERP = HOERP[~(HOERP['ERP Uniview Closure Number'].isna())]
HOERP = HOERP.sort_values(['Handover meeting date'], ascending = True).groupby(['ERP Uniview Closure Number']).head(1)

HOERP.rename({'ERP Uniview Closure Number':'Closure Number','ERP Handover Status':'ERPHOStatus','Handover meeting date':'ERPHODate',
              'Total Sold flats ':'Sold Flats (ERP)', 'Total Occupied Flats' :'Occupied Flats (ERP)'},inplace=True,axis=1)
HOERP['Sold Flats (ERP)'] = pd.to_numeric(HOERP['Sold Flats (ERP)'], errors = 'coerce')

closuresRaw = reduce(lambda left,right: pd.merge(left,right, on=['Closure Number'], how='left'),[closuresRaw,HOVMS,HOERP])
closuresRaw.rename({'Timestamp_x':'VMSHOFilledDate', 'Email Address_x':'VMSHOFilledEmail', 'Specify the HANDOVER MEETING ID_x':'VMSHOMeetingID',
                   'Timestamp_y':'ERPHOFilledDate', 'Email Address_y':'ERPHOFilledEmail', 'Specify the HANDOVER MEETING ID_y':'ERPHOMeetingID'}, inplace = True, axis = 1)
closuresRaw[['VMSHOFilledDate','ERPHOFilledDate']] = closuresRaw[['VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateFormat)

closuresRaw.loc[(closuresRaw['Sales Product'] == 'ERP only') & (closuresRaw['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Sold"
closuresRaw.loc[(closuresRaw['Sales Product'] == 'VMS only') & (closuresRaw['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Sold"

closuresRaw.loc[(closuresRaw['Sales Product'].isin({'VMS only','Both VMS + ERP'})) &  (closuresRaw['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Filled"
closuresRaw.loc[(closuresRaw['Sales Product'].isin({'ERP only','Both VMS + ERP'})) &  (closuresRaw['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Filled"

closuresRaw.loc[(closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] != "Positive"),'HO Product'] = 'VMS only'
closuresRaw.loc[(~closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] == "Positive"),'HO Product'] = 'ERP only'
closuresRaw.loc[(closuresRaw['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closuresRaw['ERPHOStatus'] == "Positive"),'HO Product'] = 'Both VMS + ERP'

closuresRaw.loc[(~closuresRaw['Sold Flats (VMS)'].isna()) & (~closuresRaw['VMS Sold Flats'].isna()),'VMS Sold Flats Mismatch'] = closuresRaw['VMS Sold Flats'] - closuresRaw['Sold Flats (VMS)']
closuresRaw.loc[(~closuresRaw['Sold Flats (ERP)'].isna()) & (~closuresRaw['Sold Flats'].isna()),'ERP Sold Flats Mismatch'] = closuresRaw['Sold Flats'] - closuresRaw['Sold Flats (ERP)']

#########################
closuresRaw.loc[((closuresRaw['HO Product'] == "Both VMS + ERP") & (closuresRaw['Sales Product'] != "Both VMS + ERP")), 'WrongHOProduct'] = True

#########################

closuresRaw.loc[(~closuresRaw['VMSHOFilledDate'].isna()), 'VMSHOClosureFilledTAT'] = (closuresRaw['VMSHOFilledDate'] - closuresRaw['Closure Form Filled Date']).dt.days
closuresRaw.loc[(~closuresRaw['ERPHOFilledDate'].isna()), 'ERPHOClosureFilledTAT'] = (closuresRaw['ERPHOFilledDate'] - closuresRaw['Closure Form Filled Date']).dt.days

closuresRaw.loc[(~closuresRaw['VMSHODate'].isna()), 'VMSHOClosureTAT'] = (closuresRaw['VMSHODate'] - closuresRaw['Closure Date']).dt.days
closuresRaw.loc[(~closuresRaw['ERPHODate'].isna()), 'ERPHOClosureTAT'] = (closuresRaw['ERPHODate'] - closuresRaw['Closure Date']).dt.days


closuresRaw = closuresRaw[['Closure Type', 'Closure Date', 'Original Closure Number',
       'Closure Number', 'Closure Form Filled Date','Closure Form Filled Time','Closure Form Filled By',
       'Team Type', 'Lead Type', 'Closure Done By (Primary Email)',
       'Closure Done By (Secondary Email)', 'Society Name (Sales)',
       'Society Type', 'Property Type', 'Closure City', 
       'Sales Product','HO Product','VMSTrialPeriod',
       'ERPTrialPeriod', 'Total Flats', 'Actual Flats', 'Sold Flats', 'VMS Sold Flats',            
       'Primary BD City', 'Primary BD Role', 'Primary BD Designation','Primary RM Mail',
       'Secondary BD City', 'Secondary BD Role', 'Secondary BD Designation', 'Secondary RM Mail',
       'MigrationMCConfirmation', 'Liked Features Form Filled', 'HO Meeting ID', 'ASP Name',
       'Closure Form Filling TAT','WrongPrimaryBDMail', 'WrongSecondaryBDMail','WrongHOProduct','CrossCityClosure', 'DuplicateClosureNumber', 
       'WrongOriginalClosureNumber', 'ClosureActivity',         
       'VMSHOStatus','ERPHOStatus','VMS Sold Flats Mismatch','ERP Sold Flats Mismatch',
       'VMSHOClosureTAT','ERPHOClosureTAT', 'VMSHOClosureFilledTAT','ERPHOClosureFilledTAT',
       'VMSHOFilledDate', 'VMSHOFilledEmail', 'VMSHOMeetingID', 'VMSHODate', 'Occupied Flats (VMS)', 'Sold Flats (VMS)', 
       'ERPHOFilledDate', 'ERPHOFilledEmail', 'ERPHOMeetingID', 'ERPHODate', 'Occupied Flats (ERP)', 'Sold Flats (ERP)', 
       'Closure Proof Link', 'Premimum Factor Proof Link',
       'Competition Replaced', 'Competition Proof Link (VMS)','Competition Proof Link (ERP)',
       'VMS Competitor being replaced', 'ERP Competitor being replaced',
       'Remarks on MC (if any)'
       ]]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [ ]:
spreadSheetId = "1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("closureMasterRaw")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, closuresRaw)

ws = wb.worksheet("HOMasterRaw")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, HORaw[HORaw['Timestamp'] >= pd.to_datetime('01 August 2021')])

#Read Corrected Data

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk/edit#gid=1791988704')
sheet =  sheet.get_worksheet(2)
closures = pd.DataFrame(sheet.get_all_records())
closures[['Closure Form Filled Time']] = closures[['Closure Form Filled Time']].apply(changeDateTimeFormat)
deltaClosures = closuresRaw[closuresRaw['Closure Form Filled Time'] > closures['Closure Form Filled Time'].max()]
closures = closures.append(deltaClosures)
# closures.drop({'Closure Form Filled Time'}, inplace = True, axis = 1)
# closures = pd.merge(closures, closuresRaw[['Closure Number','Closure Form Filled Time']], on = 'Closure Number', how = 'left')

closures = pd.merge(closures, HR_Sales[['Emp Email ID','Emp City','Role']], left_on = 'Closure Done By (Primary Email)', right_on = 'Emp Email ID', how = 'left')
closures.drop({'ClosureActivity','DuplicateClosureNumber','WrongHOProduct','WrongOriginalClosureNumber','Primary BD City','Primary BD Role', 'Secondary BD City', 'Secondary BD Role'}, inplace = True, axis = 1)
closures.rename({'Emp City':'Primary BD City','Role':'Primary BD Role'}, inplace = True, axis = 1)
closures.drop({'Emp Email ID'}, inplace = True, axis = 1)

closures['Sold Flats'] = pd.to_numeric(closures['Sold Flats'], errors = 'coerce')
closures['VMS Sold Flats'] = pd.to_numeric(closures['VMS Sold Flats'], errors = 'coerce')
closures.loc[ (closures['Sales Product'].isin({'VMS only','Both VMS + ERP'}) & (closures['VMS Sold Flats'].isna())), 'VMS Sold Flats'] = closures['Sold Flats']

###########################

closures.loc[closures['Primary BD City'].isna(),'WrongPrimaryBDMail'] = True

closures = pd.merge(closures, HR_Sales[['Emp Email ID','Emp City','Role']], left_on = 'Closure Done By (Secondary Email)', right_on = 'Emp Email ID', how = 'left')
closures.rename({'Emp City':'Secondary BD City','Role':'Secondary BD Role'}, inplace = True, axis = 1)
closures.drop({'Emp Email ID'}, inplace = True, axis = 1)

closures.loc[(closures['Secondary BD City'].isna()) & (closures['Closure Done By (Secondary Email)'] != ""),'WrongSecondaryBDMail'] = True

###########################

#closures = closures.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number','Closure Number']).head(5)

closuresDuplicateClosureNumber = closures[closures['Closure Number'].duplicated()]['Closure Number'].drop_duplicates()
closures.loc[closures['Closure Number'].isin(closuresDuplicateClosureNumber),'DuplicateClosureNumber'] = True

###########################
closures[['Closure Date','Closure Form Filled Date','VMSHOFilledDate','VMSHODate','ERPHODate','ERPHOFilledDate']] = closures[['Closure Date','Closure Form Filled Date','VMSHOFilledDate','VMSHODate','ERPHODate','ERPHOFilledDate']].apply(changeDateFormat)
closures.loc[:,'Closure Form Filling TAT'] = (closures['Closure Form Filled Date'] - closures['Closure Date']).dt.days

###########################

closures.loc[:,'CrossCityClosure'] = (closures['Closure City'] != closures['Primary BD City']) | ( (~closures['Secondary BD City'].isna()) & (closures['Closure City'] != closures['Secondary BD City']))
closures.loc[closures['CrossCityClosure'] == False,'CrossCityClosure'] = ""
###########################

closures.loc[(closures['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})) & (closures['Closure Number'] == closures['Original Closure Number']),'WrongOriginalClosureNumber'] = True

###########################

closuresActivity = closures[closures['Closure Type'].isin({'Upsell (VMS/ERP)','Winback'})]

closuresActivity = closures.sort_values(['Closure Date'], ascending = True).groupby(['Original Closure Number']).head(2)
closuresActivity = closuresActivity[(closuresActivity['Original Closure Number'].duplicated(keep= False)) & (closuresActivity['Original Closure Number']!= "")]

closuresActivity.loc[:, 'Closure Month'] = pd.to_datetime(closuresActivity['Closure Date']).dt.month
closuresActivity['Months Since Last Closure'] = closuresActivity.groupby(['Original Closure Number'])['Closure Month'].transform(lambda x: x.diff()) 
closuresActivity['Days Since Last Closure'] = closuresActivity.groupby(['Original Closure Number'])['Closure Date'].transform(lambda x: x.diff().dt.days) 

closuresActivity.sort_index(inplace=True)

closuresActivity.loc[closuresActivity['Days Since Last Closure']<=90, 'ClosureActivity'] = 'Within 90 days'
closuresActivity.loc[closuresActivity['Days Since Last Closure']<=35, 'ClosureActivity'] = 'Within 35 days'
closuresActivity.loc[closuresActivity['Months Since Last Closure']==0, 'ClosureActivity'] = 'Within Month Closure Activity'

closuresActivity = closuresActivity[~closuresActivity['ClosureActivity'].isna()][['Original Closure Number','Closure Number','ClosureActivity']].drop_duplicates()
closures = pd.merge(closures,closuresActivity, on = ['Original Closure Number','Closure Number'], how = 'left')


closures = closures[['Closure Type', 'Closure Date', 'Original Closure Number',
       'Closure Number', 'Closure Form Filled Date', 'Closure Form Filled Time','Closure Form Filled By',
       'Team Type', 'Lead Type', 'Closure Done By (Primary Email)',
       'Closure Done By (Secondary Email)', 'Society Name (Sales)',
       'Society Type', 'Property Type', 'Closure City', 
       'Sales Product','HO Product','VMSTrialPeriod',
       'ERPTrialPeriod', 'Total Flats', 'Actual Flats', 'Sold Flats', 'VMS Sold Flats',            
       'Primary BD City', 'Primary BD Role',
       'Secondary BD City', 'Secondary BD Role',
       'MigrationMCConfirmation', 'Liked Features Form Filled', 'HO Meeting ID', 'ASP Name',
       'Closure Form Filling TAT','WrongPrimaryBDMail', 'WrongSecondaryBDMail','CrossCityClosure', 'DuplicateClosureNumber', 
       'WrongOriginalClosureNumber', 'ClosureActivity',                
       'Closure Proof Link', 'Premimum Factor Proof Link',
       'Competition Replaced','Competition Proof Link (VMS)', 'Competition Proof Link (ERP)', 
       'VMS Competitor being replaced', 'ERP Competitor being replaced',
       'Remarks on MC (if any)']]



sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk/edit#gid=1791988704')
sheet =  sheet.get_worksheet(3)
HO = pd.DataFrame(sheet.get_all_records())

HO[['Timestamp']] = HO[['Timestamp']].apply(changeDateTimeFormat)
deltaHOs = HORaw[HORaw['Timestamp'] > HO['Timestamp'].max()]
HO = HO.append(deltaHOs)

HO = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date','Actual Society Name', 'Closure Product', 'Closure Type', 'Original Closure Number',      
       'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status',
       'ERP Uniview Closure Number', 'ERP Handover Status']]

HO[['Handover meeting date']] = HO[['Handover meeting date']].apply(changeDateFormat)
HO['VMS Uniview Closure Number'] = pd.to_numeric(HO['VMS Uniview Closure Number'])
HO['ERP Uniview Closure Number'] = pd.to_numeric(HO['ERP Uniview Closure Number'])


HOVMS = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'VMS Uniview Closure Number', 'VMS Handover Status']]

HOVMS = HOVMS[~(HOVMS['VMS Uniview Closure Number'].isna())]
HOVMS = HOVMS.sort_values(['Handover meeting date'], ascending = True).groupby(['VMS Uniview Closure Number']).head(1)

HOVMS.rename({'VMS Uniview Closure Number':'Closure Number','VMS Handover Status':'VMSHOStatus','Handover meeting date':'VMSHODate',
              'Total Sold flats ':'Sold Flats (VMS)', 'Total Occupied Flats' :'Occupied Flats (VMS)'},inplace=True,axis=1)
HOVMS['Sold Flats (VMS)'] = pd.to_numeric(HOVMS['Sold Flats (VMS)'], errors = 'coerce')

HOERP = HO[['Timestamp', 'Email Address', 'Specify the HANDOVER MEETING ID',
       'Handover meeting date', 'Total Occupied Flats', 'Total Sold flats ',
       'ERP Uniview Closure Number','ERP Handover Status']]
HOERP = HOERP[~(HOERP['ERP Uniview Closure Number'].isna())]
HOERP = HOERP.sort_values(['Handover meeting date'], ascending = True).groupby(['ERP Uniview Closure Number']).head(1)

HOERP.rename({'ERP Uniview Closure Number':'Closure Number','ERP Handover Status':'ERPHOStatus','Handover meeting date':'ERPHODate',
              'Total Sold flats ':'Sold Flats (ERP)', 'Total Occupied Flats' :'Occupied Flats (ERP)'},inplace=True,axis=1)
HOERP['Sold Flats (ERP)'] = pd.to_numeric(HOERP['Sold Flats (ERP)'], errors = 'coerce')

closures = reduce(lambda left,right: pd.merge(left,right, on=['Closure Number'], how='left'),[closures,HOVMS,HOERP])
closures.rename({'Timestamp_x':'VMSHOFilledDate', 'Email Address_x':'VMSHOFilledEmail', 'Specify the HANDOVER MEETING ID_x':'VMSHOMeetingID',
                   'Timestamp_y':'ERPHOFilledDate', 'Email Address_y':'ERPHOFilledEmail', 'Specify the HANDOVER MEETING ID_y':'ERPHOMeetingID'}, inplace = True, axis = 1)
closures[['VMSHOFilledDate','ERPHOFilledDate']] = closures[['VMSHOFilledDate','ERPHOFilledDate']].apply(changeDateFormat)

closures.loc[(closures['Sales Product'] == 'ERP only') & (closures['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Sold"
closures.loc[(closures['Sales Product'] == 'VMS only') & (closures['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Sold"

closures.loc[(closures['Sales Product'].isin({'VMS only','Both VMS + ERP'})) &  (closures['VMSHOStatus'].isna()) , 'VMSHOStatus'] = "Not Filled"
closures.loc[(closures['Sales Product'].isin({'ERP only','Both VMS + ERP'})) &  (closures['ERPHOStatus'].isna()) , 'ERPHOStatus'] = "Not Filled"

closures.loc[(closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] != "Positive"),'HO Product'] = 'VMS only'
closures.loc[(~closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] == "Positive"),'HO Product'] = 'ERP only'
closures.loc[(closures['VMSHOStatus'].isin({"Positive","Positive Relaxed"})) & (closures['ERPHOStatus'] == "Positive"),'HO Product'] = 'Both VMS + ERP'

closures.loc[(~closures['Sold Flats (VMS)'].isna()) & (~closures['VMS Sold Flats'].isna()),'VMS Sold Flats Mismatch'] = closures['VMS Sold Flats'] - closures['Sold Flats (VMS)']
closures.loc[(~closures['Sold Flats (ERP)'].isna()) & (~closures['Sold Flats'].isna()),'ERP Sold Flats Mismatch'] = closures['Sold Flats'] - closures['Sold Flats (ERP)']

#########################
closures.loc[((closures['HO Product'] == "Both VMS + ERP") & (closures['Sales Product'] != "Both VMS + ERP")), 'WrongHOProduct'] = True

#########################

closures.loc[(~closures['VMSHOFilledDate'].isna()), 'VMSHOClosureFilledTAT'] = (closures['VMSHOFilledDate'] - closures['Closure Form Filled Date']).dt.days
closures.loc[(~closures['ERPHOFilledDate'].isna()), 'ERPHOClosureFilledTAT'] = (closures['ERPHOFilledDate'] - closures['Closure Form Filled Date']).dt.days

closures.loc[(~closures['VMSHODate'].isna()), 'VMSHOClosureTAT'] = (closures['VMSHODate'] - closures['Closure Date']).dt.days
closures.loc[(~closures['ERPHODate'].isna()), 'ERPHOClosureTAT'] = (closures['ERPHODate'] - closures['Closure Date']).dt.days

closures.loc[:,'Closure Month'] = pd.to_datetime(closures['Closure Date']).dt.month

closures.loc[closures['Closure City'].isin({'Kolkata','Ahmedabad','Nagpur','Jaipur','Kochi'}), 'City Type'] = 'Emerging City'
closures.loc[~closures['Closure City'].isin({'Kolkata','Ahmedabad','Nagpur','Jaipur','Kochi'}), 'City Type'] = 'Main City'
closures.loc[closures['Closure City'] == 'Other City (NBH Non-Operational Cities)', 'City Type'] = 'Other City'

closures['monthDays']=pd.to_datetime(closures['Closure Date']).dt.day
closures['year']=pd.to_datetime(closures['Closure Date']).dt.year
closures['month']=pd.to_datetime(closures['Closure Date']).dt.month

def FCMultiplerVMS(row):
  if row['monthDays']>=7 and row['monthDays']<=20 and row['Sales Product']!="ERP only" and row['year']==2022 and row['month']>=1:
    return row['VMS Sold Flats']*1.2
  if row['monthDays']>=26 and row['Sales Product']!="ERP only" and row['year']==2022 and row['month']>=1:
    return row['VMS Sold Flats']*1
  else:
    return row['VMS Sold Flats']


def FCMultiplerERP(row):
  if row['monthDays']>=7 and row['monthDays']<=20 and row['Sales Product']!="VMS only" and row['year']==2022 and row['month']>=1:
    return row['Sold Flats']*1.2
  if row['monthDays']>=26 and row['Sales Product']!="VMS only" and row['year']==2022 and row['month']>=1:
    return row['Sold Flats']*1
  else:
    return row['Sold Flats']


closures["VMSFlatsNew"]=closures.apply(FCMultiplerVMS,axis=1)
closures["ERPFlatsNew"]=closures.apply(FCMultiplerERP,axis=1)


##### Builder - Under Construction Flat multiplier #######

closures['VMSFlatsNew'] = closures.apply(lambda x: 0.45*x['VMSFlatsNew'] if (
                      (x['Sales Product']!="ERP only")
                      & (x["Property Type"] == 'Builder - Under Construction')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      ) else x['VMSFlatsNew'], axis=1
              )

closures['ERPFlatsNew'] = closures.apply(lambda x: 0.45*x['ERPFlatsNew'] if (
                      (x['Sales Product']!="VMS only")
                      & (x["Property Type"] == 'Builder - Under Construction')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      ) else x['ERPFlatsNew'], axis=1
              )


##### Builder - Newly Construction Flat multiplier #######

closures['Occupied Flats (VMS)'] = closures['Occupied Flats (VMS)'].fillna(0)
closures['Occupied Flats (ERP)'] = closures['Occupied Flats (ERP)'].fillna(0)

closures['VMSFlatsNew'] = closures.apply(lambda x: 0.75*x['VMSFlatsNew'] if (
                      (x['Sales Product']!="ERP only")
                      & (x["Property Type"] == 'Builder - Newly Constructed')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      & (x["Occupied Flats (VMS)"] > 0 )
                      ) else x['VMSFlatsNew'], axis=1
              )

closures['VMSFlatsNew'] = closures.apply(lambda x: 0.65*x['VMSFlatsNew'] if (
                      (x['Sales Product']!="ERP only")
                      & (x["Property Type"] == 'Builder - Newly Constructed')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      & (x["Occupied Flats (VMS)"] == 0 )
                      ) else x['VMSFlatsNew'], axis=1
              )

closures['ERPFlatsNew'] = closures.apply(lambda x: 0.75*x['ERPFlatsNew'] if (
                      (x['Sales Product']!="VMS only")
                      & (x["Property Type"] == 'Builder - Newly Constructed')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      & (x["Occupied Flats (ERP)"] > 0 )
                      ) else x['ERPFlatsNew'], axis=1
              )

closures['ERPFlatsNew'] = closures.apply(lambda x: 0.65*x['ERPFlatsNew'] if (
                      (x['Sales Product']!="VMS only")
                      & (x["Property Type"] == 'Builder - Newly Constructed')
                      & ((x['Closure Month']) ==6 and (x['year'])==2022)
                      & (x["Occupied Flats (ERP)"] == 0 )
                      ) else x['ERPFlatsNew'], axis=1
              )



'''
closures['ERPFlatsNew'] = closures.apply(lambda x: 0.65*x['ERPFlatsNew'] if (
                      (x['Sales Product']!="VMS only")
                      & (x["Property Type"] == 'Builder - Newly Constructed')
                      & (pd.to_datetime(x['Closure Date']).date() > pd.to_datetime("1 May 2022").date())
                      & (x["Occupied Flats (ERP)"] == 0 )
                      ) else x['ERPFlatsNew'], axis=1
              )
'''

closures = closures[['Closure Type', 'Closure Date', 'Original Closure Number',
       'Closure Number', 'Closure Form Filled Date','Closure Form Filled Time','Closure Month', 'Closure Form Filled By',
       'Team Type', 'Lead Type', 'Closure Done By (Primary Email)',
       'Closure Done By (Secondary Email)', 'Society Name (Sales)',
       'Society Type', 'Property Type', 'Closure City', 'City Type',
       'Sales Product','HO Product','VMSTrialPeriod',
       'ERPTrialPeriod', 'Total Flats', 'Actual Flats', 'Sold Flats', 'VMS Sold Flats',            
       'Primary BD City', 'Primary BD Role',
       'Secondary BD City', 'Secondary BD Role',
       'MigrationMCConfirmation', 'Liked Features Form Filled', 'HO Meeting ID', 'ASP Name',
       'Closure Form Filling TAT','WrongPrimaryBDMail', 'WrongSecondaryBDMail','WrongHOProduct','CrossCityClosure', 'DuplicateClosureNumber', 
       'WrongOriginalClosureNumber', 'ClosureActivity',
       'VMSHOStatus','ERPHOStatus','VMS Sold Flats Mismatch','ERP Sold Flats Mismatch',
       'VMSHOClosureTAT','ERPHOClosureTAT', 'VMSHOClosureFilledTAT','ERPHOClosureFilledTAT',
       'VMSHOFilledDate', 'VMSHOFilledEmail', 'VMSHOMeetingID', 'VMSHODate', 'Occupied Flats (VMS)', 'Sold Flats (VMS)', 
       'ERPHOFilledDate', 'ERPHOFilledEmail', 'ERPHOMeetingID', 'ERPHODate', 'Occupied Flats (ERP)', 'Sold Flats (ERP)', 
       'Closure Proof Link', 'Premimum Factor Proof Link',
       'Competition Replaced','Competition Proof Link (VMS)', 'Competition Proof Link (ERP)',
       'VMS Competitor being replaced', 'ERP Competitor being replaced',
       'Remarks on MC (if any)','monthDays','year','VMSFlatsNew','ERPFlatsNew','month'
       ]]

In [ ]:
closures[closures["Property Type"] == 'Builder - Under Construction'][['Closure Number', 'Closure Date', 'Sales Product', "Property Type", 'Sold Flats', 'VMS Sold Flats', 'VMSFlatsNew', 'ERPFlatsNew']]

,Closure Number,Closure Date,Sales Product,Property Type,Sold Flats,VMS Sold Flats,VMSFlatsNew,ERPFlatsNew
1953,1022381,2022-03-31,VMS only,Builder - Under Construction,25.0,25.0,25.0,25.0
2051,1022598,2022-04-16,ERP only,Builder - Under Construction,300.0,NaN,NaN,360.0
2176,1022787,2022-04-22,VMS only,Builder - Under Construction,450.0,450.0,450.0,450.0
2313,1022765,2022-04-12,VMS only,Builder - Under Construction,383.0,383.0,459.6,383.0
2702,1023514,2022-05-31,VMS only,Builder - Under Construction,765.0,765.0,765.0,765.0
2708,1023510,2022-05-31,VMS only,Builder - Under Construction,705.0,705.0,705.0,705.0
2710,1023511,2022-05-31,VMS only,Builder - Under Construction,748.0,748.0,748.0,748.0
2729,1023561,2022-05-31,VMS only,Builder - Under Construction,723.0,723.0,723.0,723.0
3182,1024213,2022-06-30,VMS only,Builder - Under Construction,280.0,280.0,126.0,280.0
3206,1024251,2022-06-30,VMS only,Builder - Under Construction,186.0,186.0,83.7,186.0


In [ ]:
closures[closures["Property Type"] == 'Builder - Newly Constructed'][['Closure Number', 'Closure Date', 'Sales Product', 'Occupied Flats (VMS)','Occupied Flats (ERP)',"Property Type",
                                                                      'Sold Flats', 'VMS Sold Flats', 'VMSFlatsNew', 'ERPFlatsNew']].sort_values('Closure Date', ascending=False)

,Closure Number,Closure Date,Sales Product,Occupied Flats (VMS),Occupied Flats (ERP),Property Type,Sold Flats,VMS Sold Flats,VMSFlatsNew,ERPFlatsNew
3790,1025156,2022-08-07,VMS only,0,0,Builder - Newly Constructed,129.0,129.0,154.8,129.0
3739,1024969,2022-07-31,VMS only,44,0,Builder - Newly Constructed,55.0,55.0,55.0,55.0
3663,1024918,2022-07-31,VMS only,30,0,Builder - Newly Constructed,171.0,171.0,171.0,171.0
3656,1024906,2022-07-30,VMS only,55,0,Builder - Newly Constructed,210.0,210.0,210.0,210.0
3618,1024876,2022-07-30,VMS only,100,0,Builder - Newly Constructed,1088.0,1088.0,1088.0,1088.0
...,...,...,...,...,...,...,...,...,...,...
1900,1022310,2022-03-30,VMS only,60,0,Builder - Newly Constructed,72.0,72.0,72.0,72.0
1875,1022295,2022-03-28,VMS only,60,0,Builder - Newly Constructed,115.0,115.0,115.0,115.0
1847,1022212,2022-03-25,VMS only,45,0,Builder - Newly Constructed,140.0,140.0,140.0,140.0
1826,1022195,2022-03-23,VMS only,86,0,Builder - Newly Constructed,149.0,149.0,149.0,149.0


In [ ]:
spreadSheetId = "1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk"
wb = gc.open_by_key(spreadSheetId)

ws = wb.worksheet("closureMasterCorrected")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, closures)

ws = wb.worksheet("HOMasterCorrected")
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, HO[HO['Timestamp'] >= pd.to_datetime('01 August 2021')])

In [ ]:
closures.shape

(3986, 74)

#Sales IPs D4

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1flbVAF3DTKIJ7FXzM_S6qmZUzJRn5OPTcSPnnPauUoU/edit#gid=1861169524')
sheet =  sheet.get_worksheet(0)
PIAmultiplier = pd.DataFrame(sheet.get_all_records())

PIAmultiplier[['Closure Form Filled Time']] = PIAmultiplier[['Closure Form Filled Time']].apply(changeDateTimeFormat)
deltaClosures = closures[closures['Closure Form Filled Time'] > PIAmultiplier['Closure Form Filled Time'].max()]
PIAmultiplier = PIAmultiplier.append(deltaClosures)

In [ ]:
#Write to sheet

spreadSheetId = "1flbVAF3DTKIJ7FXzM_S6qmZUzJRn5OPTcSPnnPauUoU"
rangeName = "ClosuresMultipliers2"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
wb.values_clear("ClosuresMultipliers2!A:BQ")
df = pd.DataFrame(PIAmultiplier)
#Append data to sheet
set_with_dataframe(ws, df.iloc[:,:138])

In [ ]:
#BillMigrationList =[1018966,1018814,1019585,1019606,1020292,1020117,1021951,1021952,1022197,1022149,1021955,1022152,1021882,1022496,1022443]
#BillMigrationMultiplier = 1.25

sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1flbVAF3DTKIJ7FXzM_S6qmZUzJRn5OPTcSPnnPauUoU/edit#gid=1861169524')
sheet =  sheet.get_worksheet(0)
PIAmultiplier = pd.DataFrame(sheet.get_all_records())
PIAmultiplier = PIAmultiplier[['Closure Number','Closure Date','Premium Multiplier','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP']]
PIAmultiplier.rename({'Closure Number':'Closure Number', 'Closure Date':'Closure Date','Premium Multiplier':'Premium Multiplier','Competition Multiplier (PIA) - VMS':'Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP':'Competition Multiplier (PIA) - ERP'},inplace= True, axis = 1)

PIAmultiplier = PIAmultiplier[['Closure Number','Closure Date','Premium Multiplier','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP']]
PIAmultiplier[['Closure Date']] = PIAmultiplier[['Closure Date']].apply(changeDateFormat)
PIAmultiplier['Competition Multiplier (PIA) - VMS'] = pd.to_numeric(PIAmultiplier['Competition Multiplier (PIA) - VMS'], errors='coerce')
PIAmultiplier['Competition Multiplier (PIA) - ERP'] = pd.to_numeric(PIAmultiplier['Competition Multiplier (PIA) - ERP'], errors='coerce')
PIAmultiplier['Premium Multiplier'] = pd.to_numeric(PIAmultiplier['Premium Multiplier'], errors='coerce')
PIAmultiplier.loc[PIAmultiplier['Competition Multiplier (PIA) - VMS'].isna(), 'Competition Multiplier (PIA) - VMS'] = 1
PIAmultiplier.loc[PIAmultiplier['Competition Multiplier (PIA) - ERP'].isna(), 'Competition Multiplier (PIA) - ERP'] = 1
PIAmultiplier.loc[PIAmultiplier['Premium Multiplier'].isna(), 'Premium Multiplier'] = 1

#PIAmultiplier.loc[(PIAmultiplier['Closure Date']<= pd.to_datetime('31 October 2021').date()), 'Competition Multiplier (PIA) - ERP' ] = PIAmultiplier['Competition Multiplier (PIA) - VMS']
PIAmultiplier = PIAmultiplier.sort_values(['Premium Multiplier','Competition Multiplier (PIA) - VMS','Competition Multiplier (PIA) - ERP'], ascending = False).groupby(['Closure Number']).head(1)

closures = pd.merge(closures, PIAmultiplier, on=['Closure Number','Closure Date'], how = 'left')
closures.loc[(closures['City Type'] == 'Emerging City') & (closures['Competition Multiplier (PIA) - VMS'] == 1.5), 'Competition Multiplier (PIA) - VMS'] = 3
closures.loc[(closures['City Type'] == 'Emerging City') & (closures['Competition Multiplier (PIA) - ERP'] == 1.5), 'Competition Multiplier (PIA) - ERP'] = 3
closures.loc[closures['Premium Multiplier'].isna(), 'Premium Multiplier'] = 1
closures.loc[closures['Competition Multiplier (PIA) - VMS'].isna(), 'Competition Multiplier (PIA) - VMS'] = 1
closures.loc[closures['Competition Multiplier (PIA) - ERP'].isna(), 'Competition Multiplier (PIA) - ERP'] = 1

##############################
'''
def competitionMultiplier(row):

  VMScomp = 1
  ERPcomp = 1
  if row['Competition Replaced'] == "Yes":
    
    if row['VMS Competitor being replaced']=="MyGate" and row['VMS Sold Flats']>=300:
      VMScomp = 2
    if row['VMS Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate']:
      VMScomp = 1.5
    if row['ERP Competitor being replaced'] == "MyGate" and row['Sold Flats']>=300:
      ERPcomp = 2
    if row['ERP Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate']:
      ERPcomp = 1.5      
    
    if row['City Type'] == 'Emerging City' and row['VMS Competitor being replaced'] == "MyGate":  
        VMScomp = 3
    if row['City Type'] == 'Emerging City' and row['ERP Competitor being replaced'] == "MyGate":
        ERPcomp = 3
    return VMScomp,ERPcomp

  else:
    return VMScomp,ERPcomp

closures[['Competition Multiplier (Claimed) - VMS', 'Competition Multiplier (Claimed) - ERP']] = closures.apply(competitionMultiplier, axis = 1, result_type="expand")
'''

def competitionMultiplierVMS(row):
  if row['Competition Replaced'] == "Yes":
    
    if row['VMS Competitor being replaced']=="MyGate" and row['VMS Sold Flats']>=250 and row['City Type'] != 'Emerging City':
      return 2
    if row['VMS Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate','i-Society Manager'] and row['City Type'] != 'Emerging City':
      return 1.5
    if row['VMS Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate','i-Society Manager'] and row['City Type'] == 'Emerging City':
      return 3
    if row['VMS Competitor being replaced'] in ['None']:
      return 1
  else:
    return 1

def competitionMultiplierERP(row):
  if row['Competition Replaced'] == "Yes":
    if row['ERP Competitor being replaced'] == "MyGate" and row['Sold Flats']>=250 and row['City Type'] != 'Emerging City':
      return 2
    if row['ERP Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate','i-Society Manager'] and row['City Type'] != 'Emerging City':
      return 1.5
    if row['ERP Competitor being replaced'] in ['Apartment ADDA', 'Apna Complex', 'Common Floor Groups','MyGate','i-Society Manager'] and row['City Type'] == 'Emerging City':
      return 3
    if row['ERP Competitor being replaced'] in ['None']:
      return 1
  else:
    return 1


closures['Competition Multiplier (Claimed) - VMS'] = closures.apply(competitionMultiplierVMS, axis = 1)
closures['Competition Multiplier (Claimed) - ERP'] = closures.apply(competitionMultiplierERP, axis = 1)
##############################

def trialfactor(row):

  #VMSTrialReductionFactorD4 = 1
  #VMSTrialReductionFactorPIA = 1
  if row['Sales Product'] in ['VMS only','Both VMS + ERP']:    
    if row['Competition Multiplier (Claimed) - VMS'] > 1:
      return 1
    else:
      if row['Closure City'] in ['Bangalore','Pune']:
        if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] in ["5 Year (Limited period)","Lifetime Free offer"]:
          return 0.9
        else:
          return 1
      else:
        if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] == "1 Year (Limited period)":
          return 1
        else:
          return 0.9
  else:
    return 1

  if row['Sales Product'] in ['VMS only','Both VMS + ERP']:    
    if row['Competition Multiplier (PIA) - VMS'] > 1:
      return 1
    else:
      if row['Closure City'] in ['Bangalore','Pune']:
        if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] in ["5 Year (Limited period)","Lifetime Free offer"]:
          return 0.9
        else:
          return 1
      else:
        if row['VMSTrialPeriod'] != "" and row['VMSTrialPeriod'] == "1 Year (Limited period)":
          return 1
        else:
          return 0.9
  else:
    return 1

  #return VMSTrialReductionFactorD4, VMSTrialReductionFactorPIA

#closures[['VMSTrialReductionFactorD4','VMSTrialReductionFactorPIA']] = closures.apply(trialfactor,axis = 1, result_type = 'expand')


closures['VMSTrialReductionFactorD4']=closures.apply(trialfactor,axis = 1)
closures['VMSTrialReductionFactorPIA']=closures.apply(trialfactor,axis = 1)



##############################

productMultiplier = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Product Multiplier/Monthly Product Multiplier.csv")

def ProductWiseMultiplier(row):

  VMSProductMultiplierD4 = 0
  ERPProductMultiplierD4 = 0
  VMSProductMultiplierHO = 0
  ERPProductMultiplierHO = 0

  # bothVMSERPProductMultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'Both VMS + ERP'), 'multiplier'].values[0]
  VMSProductMultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'VMS only'), 'multiplier'].values[0]
  ERPProductMultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'ERP only'), 'multiplier'].values[0]

  if row['Sales Product'] == 'VMS only':
    VMSProductMultiplierD4 = VMSProductMultiplier*row['VMSTrialReductionFactorD4']*row['Competition Multiplier (Claimed) - VMS']
  elif row['Sales Product'] == 'ERP only':
    ERPProductMultiplierD4 = ERPProductMultiplier*row['Competition Multiplier (Claimed) - ERP']
  elif row['Sales Product'] == 'Both VMS + ERP':             
    VMSProductMultiplierD4 = VMSProductMultiplier*row['VMSTrialReductionFactorD4']*row['Competition Multiplier (Claimed) - VMS'] 
    ERPProductMultiplierD4 = ERPProductMultiplier*row['Competition Multiplier (Claimed) - ERP']

  if row['HO Product'] == 'VMS only':
    VMSProductMultiplierHO = VMSProductMultiplier*row['VMSTrialReductionFactorPIA']*row['Competition Multiplier (PIA) - VMS']
  elif row['HO Product'] == 'ERP only':
    ERPProductMultiplierHO = ERPProductMultiplier*row['Competition Multiplier (PIA) - ERP']
  elif row['HO Product'] == 'Both VMS + ERP':             
    VMSProductMultiplierHO = VMSProductMultiplier*row['VMSTrialReductionFactorPIA']*row['Competition Multiplier (PIA) - VMS'] 
    ERPProductMultiplierHO = ERPProductMultiplier*row['Competition Multiplier (PIA) - ERP']

  return VMSProductMultiplier, ERPProductMultiplier, VMSProductMultiplierD4, ERPProductMultiplierD4, VMSProductMultiplierHO, ERPProductMultiplierHO
    
  
closures[['VMSProductMultiplier', 'ERPProductMultiplier', 'VMSProductMultiplierD4', 'ERPProductMultiplierD4', 'VMSProductMultiplierHO', 'ERPProductMultiplierHO']] = closures.apply(ProductWiseMultiplier, axis=1, result_type ='expand')

######################################

#Bill Migration Multipllier
sheet  = gc.open_by_url("https://docs.google.com/spreadsheets/d/1Yoc7wCGeQk9ZN-rtb4usfcjZdcAqAzDP7vLc_Mrwvaw/")
migration_list =  pd.DataFrame(sheet.worksheet("Migration-Check").get_all_records())
migration_list = migration_list[['Uniview Closure Number', "Multiplier"]]
migration_list = migration_list.rename(columns={'Uniview Closure Number':'Closure Number', 'Multiplier':'BillMigrationMultiplier'})
migration_list = migration_list.dropna(subset=["Closure Number"]).drop_duplicates(subset=["Closure Number"])
closures = closures.merge(migration_list, how='left', on='Closure Number')
closures["BillMigrationMultiplier"] = closures["BillMigrationMultiplier"].fillna(1)

closures.loc[:, 'ERPProductMultiplierD4'] = closures['ERPProductMultiplierD4'] * closures['BillMigrationMultiplier']
closures.loc[:, 'ERPProductMultiplierHO'] = closures['ERPProductMultiplierHO'] * closures['BillMigrationMultiplier']

######################################

focusLead_multiplier = {'3X Focus Leads':3}
closures['FocusLead Multiplier']= closures['Lead Type'].map(focusLead_multiplier)
closures.loc[closures['FocusLead Multiplier'] != 3,'FocusLead Multiplier'] = 1

######################################

def city_factor(row):
    if row['Closure City'] == 'Mumbai' and (row['Primary BD City']=='South Mumbai' or row['Secondary BD City']=='South Mumbai'):
        return 1.6    
    elif row['Closure City'] == 'Navi Mumbai' and (row['Primary BD City']== 'Mumbai' or row['Secondary BD City']=='Mumbai'):
        return 1.5
    elif row['Closure City'] == 'Navi Mumbai' and (row['Primary BD City']== 'South Mumbai' or row['Secondary BD City']=='South Mumbai'):    
        return 2.5
    else:
        return 1

closures['CrossCityFactor'] = closures.apply(city_factor, axis = 1)

##############################


closures.loc[:, 'NetMultiplier VMS D4'] = closures['VMSProductMultiplierD4'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] 
closures.loc[:, 'NetMultiplier ERP D4'] = closures['ERPProductMultiplierD4'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] 
closures.loc[closures['NetMultiplier VMS D4'] > 6, 'NetMultiplier VMS D4'] = 6
closures.loc[closures['NetMultiplier ERP D4'] > 6, 'NetMultiplier ERP D4'] = 6

closures.loc[:, 'NetMultiplier D4'] = closures['NetMultiplier VMS D4'] + closures['NetMultiplier ERP D4']
closures.loc[closures['NetMultiplier D4'] > 6, 'NetMultiplier D4'] = 6

closures.loc[closures['Sales Product'] == 'VMS only','IPs D4'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS D4'] 
closures.loc[closures['Sales Product'] == 'ERP only','IPs D4'] = closures['ERPFlatsNew'] * closures['NetMultiplier ERP D4']
closures.loc[((closures['Sales Product'] == 'Both VMS + ERP') & (closures['NetMultiplier D4'] < 6)),'IPs D4'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS D4'] + closures['ERPFlatsNew'] * closures['NetMultiplier ERP D4']
closures.loc[((closures['Sales Product'] == 'Both VMS + ERP') & (closures['NetMultiplier D4'] == 6)),'IPs D4'] = closures[['ERPFlatsNew', 'VMSFlatsNew']].max(axis=1) * closures['NetMultiplier D4'] 

##############################

closures.loc[:, 'NetMultiplier VMS HO'] = closures['VMSProductMultiplierHO'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] 
closures.loc[:, 'NetMultiplier ERP HO'] = closures['ERPProductMultiplierHO'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] 
closures.loc[closures['NetMultiplier VMS HO'] > 6, 'NetMultiplier VMS HO'] = 6
closures.loc[closures['NetMultiplier ERP HO'] > 6, 'NetMultiplier ERP HO'] = 6

closures.loc[:, 'NetMultiplier HO'] = closures['NetMultiplier VMS HO'] + closures['NetMultiplier ERP HO']
closures.loc[closures['NetMultiplier HO'] > 6, 'NetMultiplier HO'] = 6

closures.loc[closures['HO Product'] == 'VMS only','IPs HO'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS HO'] 
closures.loc[closures['HO Product'] == 'ERP only','IPs HO'] = closures['ERPFlatsNew'] * closures['NetMultiplier ERP HO']
closures.loc[((closures['HO Product'] == 'Both VMS + ERP') & (closures['NetMultiplier HO'] < 6)),'IPs HO'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS HO'] + closures['ERPFlatsNew'] * closures['NetMultiplier ERP HO']
closures.loc[((closures['HO Product'] == 'Both VMS + ERP') & (closures['NetMultiplier HO'] == 6)),'IPs HO'] = closures[['ERPFlatsNew', 'VMSFlatsNew']].max(axis=1) * closures['NetMultiplier HO'] 


######################
##CRM ONLY CLOSURE MANUAL ADDITION
closures.loc[closures['Closure Number'] == 1019290, 'NetMultiplier HO'] = 0.75
closures.loc[closures['Closure Number'] == 1019290, 'IPs HO'] = closures['NetMultiplier HO'] * closures['Sold Flats']

##############################

closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'PrimaryIPsHO'] = closures['IPs HO']
closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'SecondaryIPsHO'] = 0
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'PrimaryIPsHO'] = closures['IPs HO']/2
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'SecondaryIPsHO'] = closures['IPs HO']/2

##############################

EmpIPs1 = closures.groupby(['Closure Done By (Primary Email)','Closure Month']).agg({'PrimaryIPsHO':'sum'}).reset_index()
EmpIPs2 = closures.groupby(['Closure Done By (Secondary Email)','Closure Month']).agg({'SecondaryIPsHO':'sum'}).reset_index()
EmpIPs2 = EmpIPs2[EmpIPs2['Closure Done By (Secondary Email)'] != ""]

List =  pd.DataFrame({'Emp Mail ID': (pd.Series(EmpIPs1['Closure Done By (Primary Email)'].unique()).append(to_append = [pd.Series(EmpIPs2['Closure Done By (Secondary Email)'].unique())]).unique())})
EmpIPs1 = pd.merge(List,EmpIPs1, left_on = 'Emp Mail ID', right_on = "Closure Done By (Primary Email)",  how = 'left')
EmpIPs2 = pd.merge(List,EmpIPs2, left_on = 'Emp Mail ID', right_on = "Closure Done By (Secondary Email)",  how = 'left')

EmpIPsHO = pd.merge(EmpIPs1,EmpIPs2, on = ['Emp Mail ID','Closure Month'],  how = 'outer')
EmpIPsHO = EmpIPsHO[~EmpIPsHO['Closure Month'].isna()]

def sumIPsHO(row):  

  if ~pd.isna(row['PrimaryIPsHO']) and (pd.isna(row['SecondaryIPsHO'])):
    return row['PrimaryIPsHO']
  elif pd.isna(row['PrimaryIPsHO']) and (~pd.isna(row['SecondaryIPsHO'])):
    return row['SecondaryIPsHO']
  elif (~pd.isna(row['PrimaryIPsHO'])) and (~pd.isna(row['SecondaryIPsHO'])):
    return (row['PrimaryIPsHO'] + row['SecondaryIPsHO']) 
            
EmpIPsHO['IP (HO)'] = EmpIPsHO.apply(sumIPsHO, axis = 1)

HRcurrent = HR_Sales[~((HR_Sales['Emp Status'] == 'Inactive') & (HR_Sales['LWD'] <= pd.to_datetime("2021-08-01").date()))]

EmpIPsHO = EmpIPsHO[['Emp Mail ID','Closure Month','IP (HO)']]
EmpIPsHO = pd.merge(EmpIPsHO, HRcurrent , left_on = "Emp Mail ID", right_on = 'Emp Email ID', how = 'left')


ManagerIP = EmpIPsHO.groupby(['Manager Email ID']).agg({'IP (HO)':'sum'}).reset_index()
ManagerIP.rename({'Manager Email ID':'Emp Email ID', 'IP (HO)':'IP (HO) - Team'}, inplace = True, axis = 1)

EmpIPsHO = pd.merge(EmpIPsHO, ManagerIP, on = 'Emp Email ID', how = 'left')
EmpIPsHO.loc[~EmpIPsHO['IP (HO) - Team'].isna(), 'IP (HO)'] = EmpIPsHO['IP (HO)'] + EmpIPsHO['IP (HO) - Team']

In [ ]:
#PIAmultiplier['Closure Number']=1020171
spreadSheetId="1flbVAF3DTKIJ7FXzM_S6qmZUzJRn5OPTcSPnnPauUoU"
rangeName="Multiplier"
wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()
df = pd.DataFrame(PIAmultiplier)
set_with_dataframe(ws, df)

In [ ]:
closures[closures['Closure Number'] == 1023903]

,Closure Type,Closure Date,Original Closure Number,Closure Number,Closure Form Filled Date,Closure Form Filled Time,Closure Month,Closure Form Filled By,Team Type,Lead Type,...,NetMultiplier VMS D4,NetMultiplier ERP D4,NetMultiplier D4,IPs D4,NetMultiplier VMS HO,NetMultiplier ERP HO,NetMultiplier HO,IPs HO,PrimaryIPsHO,SecondaryIPsHO
2983,New Closure,2022-06-17,1023903.0,1023903,2022-06-21,2022-06-21 11:36:51,6,mdsabah.madathody@nobroker.in,B2C Team,Central,...,1.875,2.34375,4.21875,318.9375,1.875,2.34375,4.21875,318.9375,159.46875,159.46875


In [ ]:
EmpIPsHO[EmpIPsHO['Emp Mail ID']=='bhawya.kaur@nobroker.in']

,Emp Mail ID,Closure Month,IP (HO),Emp Code,Emp Email ID,Employee Name,Emp City,Department,Designation,Role,Manager Email ID,Emp Status,DOJ,DOR,LWD,On Field Status,In PIP/ On Notice Status,Source,IP (HO) - Team
263,bhawya.kaur@nobroker.in,1.0,20508.5675,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
264,bhawya.kaur@nobroker.in,2.0,21476.2475,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
265,bhawya.kaur@nobroker.in,3.0,21887.8175,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
266,bhawya.kaur@nobroker.in,4.0,24114.4675,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
267,bhawya.kaur@nobroker.in,5.0,28322.3175,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
268,bhawya.kaur@nobroker.in,6.0,26035.4175,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
269,bhawya.kaur@nobroker.in,7.0,23667.6675,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
270,bhawya.kaur@nobroker.in,8.0,20843.4175,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
271,bhawya.kaur@nobroker.in,9.0,20455.7675,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675
272,bhawya.kaur@nobroker.in,10.0,22125.7675,NBTS11565,bhawya.kaur@nobroker.in,BHAWYA KAUR,Bangalore,Sales,Assistant Area Sales Manager,AASM,rohith.patil@nobroker.in,Active,2021-06-28,2021-07-29,NaT,On Field,,Referral,20321.7675


In [ ]:
closures[closures['Closure Done By (Primary Email)']=='bhawya.kaur@nobroker.in']

,Closure Type,Closure Date,Original Closure Number,Closure Number,Closure Form Filled Date,Closure Form Filled Time,Closure Month,Closure Form Filled By,Team Type,Lead Type,...,NetMultiplier VMS D4,NetMultiplier ERP D4,NetMultiplier D4,IPs D4,NetMultiplier VMS HO,NetMultiplier ERP HO,NetMultiplier HO,IPs HO,PrimaryIPsHO,SecondaryIPsHO
110,New Closure,2021-08-22,1017003.0,1017003,2021-08-22,NaT,8,naresha.v@nobroker.in,B2C Team,City,...,0.800,0.0000,0.8000,32.0000,0.800,0.0000,0.8000,32.000,32.000,0.000
112,New Closure,2021-08-22,1017002.0,1017002,2021-08-25,2021-08-25 16:09:43,8,naresha.v@nobroker.in,B2C Team,City,...,1.200,1.9125,3.1125,158.7375,0.800,1.2750,2.0750,105.825,105.825,0.000
378,Upsell (VMS/ERP),2021-09-26,1017003.0,1017944,2021-09-27,2021-09-27 22:32:51,9,naresha.v@nobroker.in,B2C Team,City,...,0.000,1.2750,1.2750,51.0000,0.000,1.2750,1.2750,51.000,51.000,0.000
397,New Closure,2021-09-29,1017961.0,1017961,2021-09-30,2021-09-30 12:51:37,9,naresha.v@nobroker.in,B2C Team,City,...,0.800,1.2750,2.0750,83.0000,0.800,1.2750,2.0750,83.000,83.000,0.000
561,New Closure,2021-10-21,1019110.0,1019110,2021-10-24,2021-10-24 21:16:22,10,naresha.v@nobroker.in,B2C Team,City,...,1.500,1.5000,3.0000,186.0000,1.000,1.0000,2.0000,124.000,124.000,0.000
592,New Closure,2021-10-24,1019168.0,1019168,2021-10-25,2021-10-25 15:03:40,10,naresha.v@nobroker.in,B2C Team,City,...,2.000,0.0000,2.0000,6720.0000,1.000,0.0000,1.0000,3360.000,1680.000,1680.000
771,New Closure,2021-11-05,1019434.0,1019434,2021-11-20,2021-11-20 15:23:51,11,naresha.v@nobroker.in,B2C Team,City,...,1.000,0.0000,1.0000,82.0000,1.000,0.0000,1.0000,82.000,82.000,0.000
863,New Closure,2021-11-30,1020017.0,1020017,2021-11-30,2021-11-30 22:43:25,11,naresha.v@nobroker.in,B2C Team,City,...,1.500,1.5000,3.0000,153.0000,1.000,1.0000,2.0000,102.000,102.000,0.000
982,New Closure,2021-12-14,1020310.0,1020310,2021-12-17,2021-12-17 09:52:32,12,naresha.v@nobroker.in,B2C Team,Central,...,1.500,1.0000,2.5000,310.0000,1.000,1.0000,2.0000,248.000,248.000,0.000
1081,New Closure,2021-12-26,1020267.0,1020267,2021-12-28,2021-12-28 10:45:23,12,naresha.v@nobroker.in,B2C Team,Central,...,1.500,1.5000,3.0000,615.0000,1.000,1.0000,2.0000,410.000,205.000,205.000


#ONB Map

In [ ]:
#sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1hEyTSW1ca3DvvyxzRx15g_XX6kcqHzsZ-zHMF6Rcs5s/edit#gid=0')
#sheet =  sheet.get_worksheet(0)
#kibanaMap = pd.DataFrame(sheet.get_all_records())

dump_name = 'Final Master Data'
kibanaMap = pd.read_csv(file_path + DumpDate + "/Master/"+ dump_name +".csv",parse_dates=True)


# kibanaMap = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/Final Master Data Cleanup - Final Data.csv', parse_dates= True)

kibanaMap.rename({'kibanaID':'Kibana ID','closureID.Uniview':'Closure Number'}, inplace = True, axis = 1)
kibanaMap = kibanaMap[['Kibana ID','Closure Number','dateSigning']]
kibanaMap.loc[:,'Closure Number'] = pd.to_numeric(kibanaMap['Closure Number'], errors = 'coerce')
kibanaMap = kibanaMap[~kibanaMap['Closure Number'].isna()]
kibanaMap = kibanaMap.sort_values('dateSigning').groupby(['Closure Number']).tail(1)
# t = kibanaMap.groupby(['Original Closure Number','Closure Date']).agg(set).reset_index()['Kibana ID'].diff().dropna().reset_index()


closures = pd.merge(closures, kibanaMap, on = ['Closure Number'], how = 'left')
#closures.loc[closures['Kibana ID'].str.replace(" ","")== "", 'Kibana ID'] = np.nan

#################################

dump_name = 'Society Creation Date'
society_creation_date = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)

society_creation_date= society_creation_date[['id', 'name','created_on']]
society_creation_date.rename({'id':'Kibana ID','created_on':'Kibana Creation Date','name':'Kibana Name'},inplace=True, axis=1)
society_creation_date[['Kibana Creation Date']] = society_creation_date[['Kibana Creation Date']].apply(changeDateFormat)
society_creation_date[['Kibana Name']] = society_creation_date[['Kibana Name']].apply(cleanText)
society_creation_date = society_creation_date.sort_values('Kibana Creation Date').groupby(['Kibana Name']).tail(1)
society_creation_date = society_creation_date[society_creation_date['Kibana Creation Date'] >= pd.to_datetime('1 July 2021').date()]

society_creation_date.rename({'Kibana ID':'Auto-Matched Kibana ID'}, inplace = True, axis = 1)

socnames = society_creation_date[['Kibana Name']].drop_duplicates()

def textMatch(row):  
  ratio = process.extract(row['Society Name (Sales)'], socnames['Kibana Name'] , limit = 1 , scorer = fuzz.ratio)
  if (ratio[0][1] >= 90):
    return ratio[0][0]
    
closures['Possible Kibana Name Match'] = closures.apply(textMatch, axis = 1)

closures = pd.merge(closures, society_creation_date[['Auto-Matched Kibana ID','Kibana Name']], left_on = 'Possible Kibana Name Match', right_on = 'Kibana Name', how = 'left')
#losures.loc[closures['Kibana ID'].isna() & ~closures['Auto-Matched Kibana ID'].isna(),'Kibana ID'] = closures['Auto-Matched Kibana ID']

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:1218: UserWarning: You are merging on int and float columns where the float values are not equal to their int representation
  UserWarning,


In [ ]:


kibanaMap[kibanaMap['Closure Number']==1019784]


,Kibana ID,Closure Number,dateSigning
11209,26081,1019784.0,2021-11-20


In [ ]:
closures[~closures['Kibana ID'].isna()].shape

(3693, 105)

#Ops Parameter Merge

#VMS

In [ ]:
DumpDate

'24 August 2022'

In [ ]:
Last7DaysDate

datetime.date(2022, 8, 17)

In [ ]:
dump_name = 'Society wise first visit'

FirstEntry = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv")
FirstEntry[['First entry time']] = FirstEntry[['First entry time']].apply(changeDateFormat)
FirstEntry.rename({'Society ID':'Kibana ID','First entry time':'First Entry Date'},inplace=True,axis=1)
FirstEntry.drop(['Society Name'], axis = 1,  inplace = True)

##################################

# dump_name = "Exclude P1"
# param = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
# mastermerge = pd.DataFrame(columns = param.columns) 
# masterNSEntry = getDump(mastermerge, dump_name, Last7DaysDate, DumpDate)
# dump_name = "Exclude P2"
# param = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv")
# mastermerge = pd.DataFrame(columns = param.columns) 
# mastermergeP2 = getDump(mastermerge, dump_name, Last7DaysDate, DumpDate)
# masterNSEntry = mastermergeP1.append(mastermergeP2)

##################################

dump_name = "Exclude P1"

param = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
mastermerge = pd.DataFrame(columns = param.columns) 
masterNSEntry = getDump(mastermerge, dump_name, Last7DaysDate, DumpDate)

masterNSEntry.rename({"society_id: Descending" : "Kibana ID","approval_type: Descending" : "Entry Approval Type",
                            "occupancy_status: Descending" : "Flat Occupancy Status","Count" : "Total Non-Staff Entry Count" }, inplace = True, axis = 1)
masterNSEntry['Total Non-Staff Entry Count'] = pd.to_numeric(masterNSEntry['Total Non-Staff Entry Count'], errors = 'coerce')
masterNSEntry['Kibana ID'] = masterNSEntry['Kibana ID'].apply(str)
masterNSEntry = masterNSEntry[['Kibana ID','DumpDate','Flat Occupancy Status','Entry Approval Type','Total Non-Staff Entry Count']]

masterNSEntry = masterNSEntry[masterNSEntry['Flat Occupancy Status'] != "VACANT"]

masterNSEntry['Entry Approval Type'].replace(to_replace=['RESIDENT','INTERCOM','DENIED_FINAL_RESIDENT','PRE_APPROVED','DENIED','DENIED_FINAL','INCORRECT','NOTIFIED'], value='RESIDENT', inplace=True)
masterNSEntry1 = masterNSEntry.groupby(['Kibana ID','DumpDate'])['Total Non-Staff Entry Count'].sum().reset_index()
masterNSEntry2 = masterNSEntry.groupby(['Kibana ID','DumpDate','Entry Approval Type'])['Total Non-Staff Entry Count'].sum().reset_index()
masterNSEntry2 = masterNSEntry2[masterNSEntry2['Entry Approval Type'] == 'RESIDENT']
masterNSEntry2.drop('Entry Approval Type', inplace = True, axis = 1)
masterNSEntry2.rename({'Total Non-Staff Entry Count':'Resident Non-Staff Entry Count'},inplace=True, axis=1)

masterNSEntry = pd.merge(masterNSEntry1, masterNSEntry2, left_on = ['Kibana ID','DumpDate'], right_on = ['Kibana ID','DumpDate'], how = 'right')

##################################

dump_name = 'Include staff'
param = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
mastermerge = pd.DataFrame(columns = param.columns) 
masterEntryTotal = getDump(mastermerge, dump_name, Last7DaysDate, DumpDate)

masterEntryTotal.rename({"society_id: Descending" : "Kibana ID","Count" : "Total Entry Count" }, inplace = True, axis = 1)
masterEntryTotal = masterEntryTotal[['Kibana ID','Total Entry Count','DumpDate']]
masterEntryTotal['Total Entry Count'] = pd.to_numeric(masterEntryTotal['Total Entry Count'].str.replace(",", ""))

masterEntry = pd.merge(masterEntryTotal, masterNSEntry, left_on = ['Kibana ID','DumpDate'], right_on = ['Kibana ID','DumpDate'], how = 'left')
masterEntry = masterEntry[['Kibana ID', 'DumpDate', 'Total Entry Count', 'Total Non-Staff Entry Count', 'Resident Non-Staff Entry Count']]
# masterEntry['RA perc'] = round(masterEntry['Resident Non-Staff Entry Count']/masterEntry['Total Non-Staff Entry Count']*100,0)

##################################


masterEntry = pd.merge(masterEntry, closures[['Kibana ID', 'Sold Flats']], left_on = ['Kibana ID'], right_on = ['Kibana ID'], how = 'right')
masterEntry.drop_duplicates(inplace = True)
masterEntry = masterEntry[~(masterEntry['Kibana ID'].isna())]

masterEntry = masterEntry.groupby('Kibana ID').agg({'Total Entry Count':'mean', 'Total Non-Staff Entry Count':'mean','Resident Non-Staff Entry Count':'sum','Sold Flats':'sum'}).reset_index()
masterEntry.rename({'Total Entry Count':'Avg Total Entry(7 Days)','Total Non-Staff Entry Count':'Avg Non-Staff Entry(7 Days)'}, inplace = True, axis = 1)

masterEntry.loc[:,'RA Perc'] = masterEntry['Resident Non-Staff Entry Count']/masterEntry['Sold Flats']*100
masterEntry.drop(columns = ['Sold Flats'], inplace = True)

#ERP

In [ ]:
Last90DaysDate=pd.to_datetime(Last90DaysDate).date()
Last90DaysDate



datetime.date(2022, 5, 26)

In [ ]:
DumpDate

'24 August 2022'

In [ ]:
dump_name = 'Society Creation Date'
society_creation_date = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)

society_creation_date= society_creation_date[['id', 'name','created_on']]
society_creation_date.rename({'id':'Kibana ID','created_on':'Kibana Creation Date','name':'Kibana Name'},inplace=True, axis=1)
society_creation_date[['Kibana Creation Date']] = society_creation_date[['Kibana Creation Date']].apply(changeDateFormat)
society_creation_date[['Kibana Name']] = society_creation_date[['Kibana Name']].apply(cleanText)
society_creation_date = society_creation_date[society_creation_date['Kibana Creation Date'] >= pd.to_datetime("01 July 2021").date()]


######################################################

dump_name = 'ERP Enabled Date'
erp_activation = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

erp_activation.rename({'society_id':'Kibana ID','created_on':'ERP Activation Date'},inplace=True, axis=1)
erp_activation[['ERP Activation Date']] = erp_activation[['ERP Activation Date']].apply(changeDateFormat)
erp_activation=erp_activation.sort_values('ERP Activation Date').groupby('Kibana ID').tail(1)

######################################################

# dump_name = 'Voucher Entry Date - Society Wise'
# BillDataUpload = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

# BillDataUpload.rename({'Society ID':'Kibana ID','Voucher Entry Date':'Bill Data Migration Date'},inplace=True, axis=1)
# BillDataUpload[['Bill Data Migration Date']] = BillDataUpload[['Bill Data Migration Date']].apply(changeDateFormat)

######################################################

# dump_name = 'Number of financial years'
# MigrationYears = pd.read_csv(file_path + start_date + "/Varshini/"+ dump_name +".csv",parse_dates=True)
# MigrationYears.rename({'tbl_society_master → sc_nb_sc_id':'Kibana ID','Count':'Migration Years'},inplace=True, axis=1)
# MigrationYears = MigrationYears[['Kibana ID','Migration Years']]
# MigrationYears = MigrationYears[MigrationYears['Migration Years'] >1]

######################################################

# dump_name = 'First Bill Creation and Approved Date'
# first_billDate = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
# first_billDate[['Created Date','Approved Date']] = first_billDate[['Created Date','Approved Date']].apply(changeDateFormat)
# first_billDate.rename({'Society id':'Kibana ID','Approved Date':'First Posted Date'},inplace=True, axis=1)
# first_billDate = first_billDate[['Kibana ID','First Posted Date']]

######################################################

dump_name = 'Bank account added and their status'
account = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

account[['created_on']]= account[['created_on']].apply(changeDateFormat)
account = account[['society_id','created_on','account_status']]
account = account.sort_values(['created_on']).groupby(['society_id']).tail(1)
account.rename({'society_id':'Kibana ID','created_on':'Bank Addition Date','account_status':'Society AccountStatus'},axis=1,inplace=True)


######################################################

dump_name = 'Vendor Bills added Societywise - 30 Days'
vendor_30days = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

vendor_30days.rename({'tbl_society_master → sc_nb_sc_id':'Kibana ID',
                      'tbl_society_master → sc_sm_name':'Society Name',
                      'Count':'Vendor Bill Added (Last 30 Days)'},inplace=True,axis=1)
vendor_30days=vendor_30days[['Kibana ID','Vendor Bill Added (Last 30 Days)']]

######################################################

dump_name = 'Total number of Receipt entries made by society'
receipt_entry = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

receipt_entry.rename({'tbl_society_master → sc_nb_sc_id':'Kibana ID',
                      'tbl_society_master → sc_sm_billing_name':'Society Name',
                      'created_on':'DumpDate',
                      'Distinct values of voucher_no':'Receipt Entries'},inplace=True,axis=1)

receipt_entry[['DumpDate']] = receipt_entry[['DumpDate']].apply(changeDateFormat)
receipt_entryCurr = receipt_entry[receipt_entry['DumpDate'] >= Last90DaysDate]

receipt_entry = receipt_entry[['Kibana ID','DumpDate','Receipt Entries']]
receipt_entry = receipt_entry.groupby(['Kibana ID']).agg({'Receipt Entries':'sum'}).reset_index()
receipt_entry.rename({'Receipt Entries':'Total Receipt Entries '},inplace=True,axis=1)

######################################################

dump_name = 'Society wise apartments with valid phone or email'
RD = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
RD.rename({'society_id':'Kibana ID','apartments':'RDcount'},axis=1,inplace=True)
RD = RD[~RD['Kibana ID'].isna()]

######################################################

dump_name = 'Device installed by society with apartment count without date'

#dump_name = 'App Download Summary V2'
Download = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/Datewise Dumps - Aug Onwards/Aug 2022/21 August 2022/Sai/Device installed by society with apartment count without date.csv")
#Download = pd.read_csv(file_path + DumpDate + "/Sai/"+ dump_name +".csv",parse_dates=True)
Download.rename({'society_id':'Kibana ID','no_of_apartments':'Download Count'},inplace=True,axis=1)
#Download.rename({'Society ID':'Kibana ID','Count':'Download Count'},inplace=True,parse_dates=True)
Download = Download[['Kibana ID','Download Count']]


######################################################

#To be checked.. Take directly 60 days daterange dump
# dump_name = 'Last 30 day transactions with bifurcation'
# param = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
# mastermerge = pd.DataFrame(columns = param.columns) 
# masterTransaction = getDump(mastermerge, dump_name, Last60DaysDate, DumpDate)

masterTransactionMaster = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/TransactionMaster/TransactionsJanOct2021.csv')
masterTransactionMaster[['Payment Date']] = masterTransactionMaster[['Payment Date']].apply(changeDateFormat)
masterTransactionMaster = masterTransactionMaster[masterTransactionMaster['Payment Date'] >= pd.to_datetime("01 August 2021").date()]

#daysDelta = 0
#DumpDate =  pd.to_datetime((datetime.today() - timedelta(days = daysDelta)).date()).strftime('%d %B %Y')
dump_name = 'Transaction Details - With Transaction Payment Date Details'
masterTransaction = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
masterTransaction[['Payment Date']] = masterTransaction[['Payment Date']].apply(changeDateFormat)
masterTransaction = masterTransaction[masterTransaction['Payment Date'] >= pd.to_datetime("01 November 2021").date()]

masterTransaction = masterTransactionMaster.append(masterTransaction)

masterTransaction = masterTransaction[['Society ID','Unique Flats', 'Transactions', 'Amount','Payment Mode','Payment Date']]
masterTransaction.rename({'Society ID':'Kibana ID','Transactions':'Payments Count','Unique Flats':'Payment Unique Flats','Amount':'Total Payment Amount in K'},inplace=True,axis=1)
masterTransaction['Total Payment Amount in K'] = (masterTransaction['Total Payment Amount in K']/1000).astype(int)
masterTransaction[['Payment Date']] = masterTransaction[['Payment Date']].apply(changeDateFormat)

masterTransaction = masterTransaction[masterTransaction['Payment Mode'] == 'Online']
masterTransaction = masterTransaction[masterTransaction['Payment Date'] >= pd.to_datetime('1 August 2021').date()]

Transaction = masterTransaction.groupby('Kibana ID').agg({'Payment Unique Flats':'sum','Total Payment Amount in K':'sum'}).reset_index()

######################################################

dump_name = 'Bill Publish Date - Society wise'
bpd = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)
bpd = bpd[bpd['Bill Type'] == 'Recurring']

bpd=bpd[['Society Name', 'Society ID', 'Start Date', 'End Date', 'Due Date',
       'Created Date', 'Approved Date', 'Bill Types', 'Bill Frequency','Valid Bill Count', 'Flat Count', 'Total Amount']]

bpd[['Start Date','End Date','Due Date','Created Date','Approved Date']] = bpd[['Start Date','End Date','Due Date','Created Date','Approved Date']].apply(changeDateFormat)

######################################################

dump_name = 'ERP - Deleted Cancelled Bills Report'
deleted = pd.read_csv(file_path + DumpDate + "/Varshini/"+ dump_name +".csv",parse_dates=True)

deleted.rename({'Deletion/Cancellation Date':'Deleted Date','Bill Type':'Bill Types','Bill Count':'Valid Bill Count','Status':'Deletion Status'},inplace=True,axis=1)
deleted=deleted[['Society Name', 'Society ID',  'Start Date', 'End Date','Due Date',
                 'Created Date','Approved Date', 'Bill Types','Bill Frequency', 'Valid Bill Count',
                 'Flat Count','Deleted Date','Deletion Status','Approval Status']]

deleted[['Start Date','End Date','Due Date','Created Date','Approved Date','Deleted Date']] = deleted[['Start Date','End Date','Due Date','Created Date','Approved Date','Deleted Date']].apply(changeDateFormat)

deleted=deleted[deleted['Approval Status']=='Approved']


bpd_deleted=bpd.append(deleted)
bpd_deleted['Deletion Status']=bpd_deleted['Deletion Status'].fillna('Not Deleted')
bpd_deleted['Bill Types']=bpd_deleted['Bill Types'].str.lower()

def bill_type_clean(row):
    if 'maint' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'mait' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'maiint' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'cam' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'e_m' in row['Bill Types']:
        return 'MaintainanceAUTOTAG'
    elif 'park' in row['Bill Types']:
        return "ParkingChargesAUTOTAG"
    elif 'water' in row['Bill Types']:
        return 'WaterAUTOTAG'
    elif 'electricity' in row['Bill Types']:
        return 'ElectricityAUTOTAG'
    elif 'utility' in row['Bill Types']:
        return 'UtilityAUTOTAG'   
    elif 'gas' in row['Bill Types']:
        return 'GasAUTOTAG'     
    else:
        return row['Bill Types']

bpd_deleted['Bill Types']=bpd_deleted.apply(bill_type_clean,axis=1)

def cleanBillFrequency(row):
    if row['Bill Frequency']>0 and row['Bill Frequency']<=12:
        return 12/12
    elif row['Bill Frequency']>20 and row['Bill Frequency']<=24:
        return 12/4
    elif row['Bill Frequency']>26 and row['Bill Frequency']<=28:
        return 12/2
    else:
        return 12/1

bpd_deleted['Bill Frequency']=bpd_deleted.apply(cleanBillFrequency,axis=1)

bpd_deleted=bpd_deleted[['Society ID', 'Created Date', 'Approved Date', 'Deleted Date','Deletion Status',
                         'Bill Frequency','Bill Types', 'Start Date', 'End Date', 'Due Date',
                         'Valid Bill Count', 'Flat Count', 'Total Amount'                         
                         ]]
bpd_deleted.rename({'Society ID':'Kibana ID','Approved Date':'Posted Date', 'Total Amount':'Total Posted Amount in K'},inplace=True,axis=1)

bpd_deleted['Total Posted Amount in K'] = (bpd_deleted['Total Posted Amount in K']/1000).round(0)

######################################################

bpd_deleted = bpd_deleted[bpd_deleted['Posted Date'] >= pd.to_datetime('1 August 2021').date()]



/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#Posting Calculations

In [ ]:
masterKibana = reduce(lambda left,right: pd.merge(left,right, on=['Kibana ID'], how='outer'),[society_creation_date,account,Transaction,RD])
masterTechPosting = pd.merge(masterKibana, bpd_deleted, left_on = ['Kibana ID'], right_on = ['Kibana ID'], how = 'left')
masterTechPosting = pd.merge(masterTechPosting, closures[['Kibana ID', 'Sold Flats']], left_on = ['Kibana ID'], right_on = ['Kibana ID'], how = 'right')
masterTechPosting.drop_duplicates(inplace = True)
masterTechPosting = masterTechPosting[~(masterTechPosting['Kibana ID'].isna())]

masterERPPosting = masterTechPosting.copy()
masterERPPosting['Sold Flats'].fillna(masterERPPosting['RDcount'], inplace=True)
masterERPPosting = masterERPPosting[~(masterERPPosting['Sold Flats'].isna())]
masterERPPosting['RDperc'] = (masterERPPosting['RDcount']/masterERPPosting['Sold Flats']*100)
masterERPPosting['Postingperc'] = (masterERPPosting['Flat Count']/masterERPPosting['Sold Flats']*100)
masterERPPosting['TransactionPerc'] = (masterERPPosting['Payment Unique Flats']/masterERPPosting['Sold Flats']*100)


conditions = [(masterERPPosting['Postingperc'].isna()),
              (masterERPPosting['Postingperc'] <= 50)]              
values = ['No Posting','Flat Posting IPR']
masterERPPosting['Flat Posting IPR'] = np.select(conditions, values)


conditions = [(masterERPPosting['RDperc'] <= 50)]
values = ['RD IPR']
masterERPPosting['RD IPR'] = np.select(conditions, values)


conditions = [(masterERPPosting['Society AccountStatus'] != 'VERIFIED')]
values = ['Account IPR']
masterERPPosting['Account IPR'] = np.select(conditions, values)

conditions = [(masterERPPosting['TransactionPerc'] <= 5)]
values = ['Transaction IPR']
masterERPPosting['Transaction IPR'] = np.select(conditions, values)


cols = ['Flat Posting IPR', 'RD IPR', 'Account IPR', 'Transaction IPR']
masterERPPosting['combined IPR'] = masterERPPosting[cols].apply(lambda row: '||'.join(row.values.astype(str)), axis=1)
masterERPPosting['Start Date1'] = pd.to_datetime(masterERPPosting[~masterERPPosting['Start Date'].isna()]['Start Date']).apply(lambda x: str(x.to_period('M')))
masterERPPosting['End Date1'] = pd.to_datetime(masterERPPosting[~masterERPPosting['End Date'].isna()]['End Date']).apply(lambda x: str(x.to_period('M')))
masterERPPosting['Bill Period'] = (masterERPPosting['Start Date1']) + ' to ' + (masterERPPosting['End Date1'])

def diff_month(row):
  return (row['End Date'].year - row['Start Date'].year) * 12 + row['End Date'].month - row['Start Date'].month  + int(round((row['End Date'].day - row['Start Date'].day)/28))

masterERPPosting['Bill Cycles Posted']=(masterERPPosting[~((masterERPPosting['End Date'].isna()) | (masterERPPosting['Start Date'].isna()))].apply(diff_month,axis=1))/masterERPPosting['Bill Frequency']

masterERPPosting.drop(['Start Date1', 'End Date1'], inplace = True, axis = 1)

# ERPPostingSummary = masterERPPosting[['Kibana ID', 'Posted Date','Deletion Status','combined IPR', 'Bill Types', 'Flat Count','Total Posted Amount in K']]
# ERPPostingSummary = ERPPostingSummary.sort_values(['Total Posted Amount in K']).groupby('Kibana ID').tail(1)

#############################################################################

masterERPPostingProper = masterERPPosting[masterERPPosting['Flat Posting IPR'] == '0']
masterERPPostingProper = masterERPPostingProper[~(masterERPPostingProper['Posted Date'].isna())]

masterERPPostingProperSummary = masterERPPostingProper.groupby(['Kibana ID','Bill Types','Bill Frequency']).agg({'Posted Date': ['count','min', 'max']}).reset_index()
masterERPPostingProper = pd.merge(masterERPPostingProper, masterERPPostingProperSummary, on = ['Kibana ID','Bill Types','Bill Frequency'], how = 'left')
masterERPPostingProper = masterERPPostingProper[~(masterERPPostingProper[('Posted Date', 'min')].isna())]

def diff_month(row):
  strt_dt = row[('Posted Date', 'min')] 
  end_dt = pd.to_datetime("today")
  return (end_dt.year - strt_dt.year) * 12 + end_dt.month - strt_dt.month + int(round((end_dt.day - strt_dt.day)/28))

masterERPPostingProper['Current Counter']= masterERPPostingProper.apply(diff_month,axis=1)
masterERPPostingProper['Current Counter'] = masterERPPostingProper['Current Counter']/masterERPPostingProper['Bill Frequency']


masterERPPostingProper.rename({('Posted Date', 'count'): 'Current Posting Counter', 'Current Counter':'Current Billing Counter',('Posted Date', 'min'):'First Posting',('Posted Date', 'max'):'Latest Posting'}, inplace = True, axis = 1)


masterERPPostingProper = masterERPPostingProper[['Kibana ID', 'Posted Date', 'Deletion Status', 'combined IPR',
       'Bill Types', 'Bill Frequency','Flat Count', 'Total Posted Amount in K','Current Billing Counter','Current Posting Counter', 'First Posting', 'Latest Posting']]

masterERPPostingProper['Posted Month'] = pd.to_datetime(masterERPPostingProper['Posted Date']).dt.month

masterERPPostingProper = masterERPPostingProper.sort_values(['Total Posted Amount in K']).groupby(['Kibana ID','Posted Month']).tail(1)
masterERPPostingProper = masterERPPostingProper.sort_values(['Posted Date']).groupby(['Kibana ID']).tail(1)
masterERPPostingProper.drop(['Posted Month'],inplace = True, axis = 1)
masterERPPostingProper.drop_duplicates(inplace = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (1 levels on the left,2 on the right)
/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


#Closure Kibana Product Parametes Merge

In [ ]:
closures = pd.merge(closures, society_creation_date[['Kibana ID','Kibana Creation Date']], on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, RD, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, Download, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, account, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, masterERPPostingProper, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, Transaction, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, vendor_30days, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, receipt_entry, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, FirstEntry, on = ['Kibana ID'], how = 'left')
closures = pd.merge(closures, masterEntry, on = ['Kibana ID'], how = 'left')

closures.loc[:,'Download Perc'] = closures['Download Count']/closures['Sold Flats']*100
closures.loc[:,'RD Perc'] = closures['RDcount']/closures['Sold Flats']*100

In [ ]:
closures

,Closure Type,Closure Date,Original Closure Number,Closure Number,Closure Form Filled Date,Closure Form Filled Time,Closure Month,Closure Form Filled By,Team Type,Lead Type,...,Total Payment Amount in K,Vendor Bill Added (Last 30 Days),Total Receipt Entries,First Entry Date,Avg Total Entry(7 Days),Avg Non-Staff Entry(7 Days),Resident Non-Staff Entry Count,RA Perc,Download Perc,RD Perc
0,New Closure,2021-08-02,1016637.0,1016637,2021-08-03,2021-08-03 15:40:06,8,akshay.wagh@nobroker.in,B2C Team,City,...,812.0,NaN,1165.0,2021-10-27,96.500,68.000,474.0,59.250000,90.000000,94.000000
1,New Closure,2021-08-03,1016640.0,1016640,2021-08-03,2021-08-03 16:59:47,8,rishabh.goel@nobroker.in,B2C Team,Central,...,NaN,NaN,NaN,2021-10-27,53.625,51.125,299.0,11.979167,68.910256,89.102564
2,Upsell (VMS/ERP),2021-08-03,4001.0,1016645,2021-08-03,2021-08-03 18:02:50,8,t.shyam@nobroker.in,B2C Team,City,...,NaN,NaN,NaN,2021-10-27,201.375,123.625,488.0,20.333333,121.000000,155.333333
3,New Closure,2021-08-04,1016636.0,1016636,2021-08-04,2021-08-04 14:58:55,8,dishant.agrawal@nobroker.in,B2C Team,City,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.000000,NaN,46.368715
4,New Closure,2021-08-04,1016655.0,1016655,2021-08-04,2021-08-04 14:23:23,8,t.shyam@nobroker.in,B2C Team,Central,...,38.0,NaN,267.0,2021-10-27,NaN,NaN,0.0,0.000000,10.714286,91.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,New Closure,2022-08-17,1025495.0,1025495,2022-08-24,2022-08-24 10:54:31,8,mdsabah.madathody@nobroker.in,B2C Team,City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5429,New Closure,2022-08-18,1024728.0,1024728,2022-08-24,2022-08-24 11:47:10,8,harshavardhan.jk@nobroker.in,B2C Team,City,...,NaN,NaN,8203.0,NaN,NaN,NaN,0.0,0.000000,NaN,NaN
5430,New Closure,2022-08-18,1025418.0,1025418,2022-08-24,2022-08-24 12:20:26,8,harshavardhan.jk@nobroker.in,B2C Team,City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5431,New Closure,2022-08-23,1025512.0,1025512,2022-08-24,2022-08-24 12:49:32,8,rajas.ghodekar@nobroker.in,B2C Team,City,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Sales D1 IP

In [ ]:
def D1ERP_Eligible(row):

  D1_ERPEligible = "junk"
  
  if row['HO Product'] in ['ERP only', 'Both VMS + ERP']:
    
    if row['Download Perc'] < 10 or str(row['combined IPR']).split('||')[0] != '0':
      D1_ERPEligible = "No"
    elif row['Download Perc'] >= 10 and str(row['combined IPR']).split('||')[0] == '0' and str(row['combined IPR']).split('||')[1] == '0' and str(row['combined IPR']).split('||')[2] == '0':
      D1_ERPEligible = "Yes"
  else:
    D1_ERPEligible = "Not Sold/HO"

  return D1_ERPEligible

closures['D1_ERPEligible'] = closures.apply(D1ERP_Eligible, axis = 1)


def downloadRDIPslab(row):
  
  RDMultiplier = 0
  if ~pd.isnull(row['RD Perc']):
    if row['RD Perc'] <40:
      RDMultiplier = 0
    elif row['RD Perc'] >= 40 and row['RD Perc'] <60:
      RDMultiplier = 0.5
    elif row['RD Perc'] >= 60 and row['RD Perc'] <80:
      RDMultiplier = 0.75
    elif row['RD Perc'] >= 80:
      RDMultiplier = 1
    
    if row['RD Perc'] >= 90 and row['Sales Product'] in ['ERP only', 'Both VMS + ERP']:
      RDMultiplier = RDMultiplier * 1.25
  else:
      RDMultiplier = 0

  DownloadMultiplier = 0
  if ~pd.isnull(row['Download Perc']):

    if row['Sold Flats'] <= 150:

      if row['Download Perc'] <10:
        DownloadMultiplier = 0
      elif row['Download Perc'] >= 10:
        DownloadMultiplier = 1

    elif row['Sold Flats'] > 150:

      if row['Download Perc'] <20:
        DownloadMultiplier = 0
      elif row['Download Perc'] >= 20 and row['Download Perc'] <30:
        DownloadMultiplier = 0.5
      elif row['Download Perc'] >= 30:
        DownloadMultiplier = 1           
  else:
      DownloadMultiplier = 0

  return max(RDMultiplier,DownloadMultiplier)  

closures['RDDownloadMultiplier'] = closures.apply(downloadRDIPslab, axis = 1)

######################################

closures.loc[:,'D1 Product'] = closures['HO Product']
closures.loc[(closures['HO Product']== 'Both VMS + ERP') & (closures['D1_ERPEligible'] == 'No'),'D1 Product'] = "VMS only"
closures.loc[(closures['HO Product']== 'ERP only') & (closures['D1_ERPEligible'] == 'No'),'D1 Product'] = "Not Eligible"

######################################

def ProductMultiplier1(row):

  VMSProductMultiplierD1 = 0
  ERPProductMultiplierD1 = 0  

  # bothVMSERPmultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'Both VMS + ERP'), 'multiplier'].values[0]
  VMSProductMultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'VMS only'), 'multiplier'].values[0]
  ERPProductMultiplier = productMultiplier.loc[(productMultiplier['year'] == row['Closure Date'].year) & (productMultiplier['month'] == row['Closure Date'].month) & (productMultiplier['product'] == 'ERP only'), 'multiplier'].values[0]

  if row['D1 Product'] == "Not Eligible":
    ERPProductMultiplierD1 = 0

  if row['D1 Product'] == 'VMS only':
    VMSProductMultiplierD1 = VMSProductMultiplier*row['VMSTrialReductionFactorPIA']*row['Competition Multiplier (PIA) - VMS']
  elif row['D1 Product'] == 'ERP only':
    ERPProductMultiplierD1 = ERPProductMultiplier*row['Competition Multiplier (PIA) - ERP']
  elif row['D1 Product'] == 'Both VMS + ERP':             
    VMSProductMultiplierD1 = VMSProductMultiplier*row['VMSTrialReductionFactorPIA']*row['Competition Multiplier (PIA) - VMS'] 
    ERPProductMultiplierD1 = ERPProductMultiplier*row['Competition Multiplier (PIA) - ERP']

  return VMSProductMultiplierD1, ERPProductMultiplierD1  
  
closures[['VMSProductMultiplierD1', 'ERPProductMultiplierD1']] = closures.apply(ProductMultiplier1, axis=1, result_type = 'expand')

######################################

closures.loc[:, 'NetMultiplier VMS D1'] = closures['VMSProductMultiplierD1'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] * closures['RDDownloadMultiplier']
closures.loc[:, 'NetMultiplier ERP D1'] = closures['ERPProductMultiplierD1'] * closures['Premium Multiplier'] * closures['CrossCityFactor'] * closures['FocusLead Multiplier'] * closures['RDDownloadMultiplier'] * closures['BillMigrationMultiplier']
closures.loc[closures['NetMultiplier VMS D1'] > 6, 'NetMultiplier VMS D1'] = 6
closures.loc[closures['NetMultiplier ERP D1'] > 6, 'NetMultiplier ERP D1'] = 6

closures.loc[:, 'NetMultiplier D1'] = closures['NetMultiplier VMS D1'] + closures['NetMultiplier ERP D1']
closures.loc[closures['NetMultiplier D1'] > 6, 'NetMultiplier D1'] = 6


closures.loc[closures['D1 Product'] == 'VMS only','IPs D1'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS D1'] 
closures.loc[closures['D1 Product'] == 'ERP only','IPs D1'] = closures['ERPFlatsNew'] * closures['NetMultiplier ERP D1']
closures.loc[((closures['D1 Product'] == 'Both VMS + ERP') & (closures['NetMultiplier D1'] < 6)),'IPs D1'] = closures['VMSFlatsNew'] * closures['NetMultiplier VMS D1'] + closures['ERPFlatsNew'] * closures['NetMultiplier ERP D1']
closures.loc[((closures['D1 Product'] == 'Both VMS + ERP') & (closures['NetMultiplier D1'] == 6)),'IPs D1'] = closures[['ERPFlatsNew', 'VMSFlatsNew']].max(axis=1) * closures['NetMultiplier D1'] 

closures.loc[closures['Closure Number'] == 1019290, 'NetMultiplier D1'] = 0.75
closures.loc[closures['Closure Number'] == 1019290, 'IPs D1'] = closures['NetMultiplier D1'] * closures['Sold Flats'] * closures['RDDownloadMultiplier']

## For Builder Under construction sites ##

closures.loc[closures['Closure Number'].isin([1024213,1024244,1024251,1024201,1024136,1024718,1023895,1024897]), 'NetMultiplier D1'] = 1 * closures['NetMultiplier HO']
closures.loc[closures['Closure Number'].isin([1024213,1024244,1024251,1024201,1024136,1024718,1023895,1024897]),'IPs D1'] = closures['NetMultiplier D1'] * closures['VMSFlatsNew']


######################################

closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'PrimaryIPsD1'] = closures['IPs D1']
closures.loc[closures['Closure Done By (Secondary Email)'] == "", 'SecondaryIPsD1'] = 0
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'PrimaryIPsD1'] = closures['IPs D1']/2
closures.loc[closures['Closure Done By (Secondary Email)'] != "", 'SecondaryIPsD1'] = closures['IPs D1']/2

######################################

EmpIPs1 = closures.groupby(['Closure Done By (Primary Email)','Closure Month']).agg({'PrimaryIPsD1':'sum'}).reset_index()
EmpIPs2 = closures.groupby(['Closure Done By (Secondary Email)','Closure Month']).agg({'SecondaryIPsD1':'sum'}).reset_index()
EmpIPs2 = EmpIPs2[EmpIPs2['Closure Done By (Secondary Email)'] != ""]

List =  pd.DataFrame({'Emp Mail ID': (pd.Series(EmpIPs1['Closure Done By (Primary Email)'].unique()).append(to_append = [pd.Series(EmpIPs2['Closure Done By (Secondary Email)'].unique())]).unique())})
EmpIPs1 = pd.merge(List,EmpIPs1, left_on = 'Emp Mail ID', right_on = "Closure Done By (Primary Email)",  how = 'left')
EmpIPs2 = pd.merge(List,EmpIPs2, left_on = 'Emp Mail ID', right_on = "Closure Done By (Secondary Email)",  how = 'left')

EmpIPsD1 = pd.merge(EmpIPs1,EmpIPs2, on = ['Emp Mail ID','Closure Month'],  how = 'outer')
EmpIPsD1 = EmpIPsD1[~EmpIPsD1['Closure Month'].isna()]

def sumIPsD1(row):  

  if ~pd.isna(row['PrimaryIPsD1']) and (pd.isna(row['SecondaryIPsD1'])):
    return row['PrimaryIPsD1']
  elif pd.isna(row['PrimaryIPsD1']) and (~pd.isna(row['SecondaryIPsD1'])):
    return row['SecondaryIPsD1']
  elif (~pd.isna(row['PrimaryIPsD1'])) and (~pd.isna(row['SecondaryIPsD1'])):
    return (row['PrimaryIPsD1'] + row['SecondaryIPsD1'])
            
EmpIPsD1['IP (D1)'] = EmpIPsD1.apply(sumIPsD1, axis = 1)

EmpIPsD1 = EmpIPsD1[['Emp Mail ID','Closure Month','IP (D1)']]

In [ ]:
closures.shape

(5433, 143)

In [ ]:
EmpIPsD1.shape

(1544, 3)

#Targets and Payout

In [ ]:
TSMTargetsSep = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Target/September 2021/TSM Targets.csv')
TSMTargetsOct = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Target/October 2021/TSM Targets.csv')
TSMTargetsNov = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Target/November 2021/TSM Targets.csv')
TSMTargetsDec = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Target/December 2021/TSM Targets.csv')
TSMTargetsJan = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Target/January 2022/TSM Targets.csv')

TSMTargetsSep = TSMTargetsSep[['Mail ID','Status','Target','Reporting supervisor Mail ID','Team']]
TSMTargetsOct = TSMTargetsOct[['Mail ID','Status','Target','Reporting supervisor Mail ID','Team']]
TSMTargetsNov = TSMTargetsNov[['Mail ID','Status','Target','Reporting supervisor Mail ID','Team']]
TSMTargetsDec = TSMTargetsDec[['Mail ID','Status','Target','Reporting supervisor Mail ID','Team']]
TSMTargetsJan = TSMTargetsJan[['Mail ID','Status','Target','Reporting supervisor Mail ID','Team']]

TSMTargetsSep.loc[:,'Closure Month'] = 9
TSMTargetsOct.loc[:,'Closure Month'] = 10
TSMTargetsNov.loc[:,'Closure Month'] = 11
TSMTargetsDec.loc[:,'Closure Month'] = 12
TSMTargetsJan.loc[:,'Closure Month'] = 1

Targets = TSMTargetsSep.append(TSMTargetsOct)
Targets = Targets.append(TSMTargetsNov)
Targets = Targets.append(TSMTargetsDec)
Targets = Targets.append(TSMTargetsJan)

Targets.rename({'Mail ID':'Emp Mail ID'}, inplace = True, axis = 1)

############################################################################

# TSMPayoutSlabsSep = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/September 2021/TSM Slab.csv')
TSMPayoutSlabsSep = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/October 2021/TSM Slab.csv')
TSMPayoutSlabsOct = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/October 2021/TSM Slab.csv')
TSMPayoutSlabsNov = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/November 2021/TSM Slab.csv')
TSMPayoutSlabsDec = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/December 2021/TSM Slab.csv')
TSMPayoutSlabsJan = pd.read_csv('/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Payout Slab/January 2022/TSM Slab.csv')

TSMPayoutSlabsSep.loc[:,'Closure Month'] = 9
TSMPayoutSlabsSep = TSMPayoutSlabsSep.head(20)
TSMPayoutSlabsOct.loc[:,'Closure Month'] = 10
TSMPayoutSlabsNov.loc[:,'Closure Month'] = 11
TSMPayoutSlabsDec.loc[:,'Closure Month'] = 12
TSMPayoutSlabsJan.loc[:,'Closure Month'] = 1

TSMPayoutSlabs = TSMPayoutSlabsSep.append(TSMPayoutSlabsOct)
TSMPayoutSlabs = TSMPayoutSlabs.append(TSMPayoutSlabsNov)
TSMPayoutSlabs = TSMPayoutSlabs.append(TSMPayoutSlabsDec)
TSMPayoutSlabs = TSMPayoutSlabs.append(TSMPayoutSlabsJan)

TSMPayoutSlabs['% Target Achieved'] = TSMPayoutSlabs['% Target Achieved'].apply(lambda x: pd.to_numeric(str(x).split("%")[0]))
TSMPayoutSlabs['Incentive %'] = TSMPayoutSlabs['Incentive %'].apply(lambda x: pd.to_numeric(str(x).split("%")[0]))

############################################################################

EmpIPs = pd.merge(EmpIPsHO, EmpIPsD1, on = ['Emp Mail ID', 'Closure Month'], how = 'left')
EmpIPs = pd.merge(EmpIPs, Targets, on = ['Emp Mail ID', 'Closure Month'], how = 'left')
EmpIPs.loc[:,'Tenure'] = (datetime.today().date() - EmpIPs['DOJ']).dt.days

EmpIPs = EmpIPs[[ 'Emp Email ID', 'Emp City', 'Role', 'Team', 'Reporting supervisor Mail ID',
       'Emp Status', 'Tenure','Source', 'In PIP/ On Notice Status','Closure Month','Target', 'IP (HO)','IP (D1)', 'IP (HO) - Team']]

EmpIPs.loc[~EmpIPs['Target'].isna(), 'Ach % HO'] = EmpIPs['IP (HO)']/EmpIPs['Target']*100
EmpIPs.loc[~EmpIPs['Target'].isna(), 'Ach % D1'] = EmpIPs['IP (D1)']/EmpIPs['Target']*100

############################################################################

def payoutCapping(row):

  if row['Role'] in ['TSM', 'SENIOR TSM']:
  
    if row['Emp City'] == 'South Mumbai':
      if row['Target']<=80 and row['Ach % HO']>100:
        AchHO = 100
      elif row['Target']>80 and row['Target']<=200  and row['Ach % HO']>200:
        AchHO = 200
      elif row['Target']>200 and row['Target']<=400  and row['Ach % HO']>400:
        AchHO = 400
      elif row['Target']>400 and row['Ach % HO']>400:
        AchHO = row['Ach % HO']
      else:
        AchHO = row['Ach % HO']
    elif row['Emp City']=='Kolkata':
      if row['Target']<=335 and row['Ach % HO']>100:
        AchHO = 100
      elif row['Target']>=335 and row['Target']<=669 and row['Ach % HO']>400:
        AchHO = 400
      else:
        AchHO = row['Ach % HO']
    else:
      if row['Target']<=200 and row['Ach % HO']>100:
        AchHO = 100
      elif row['Target']>200 and row['Target']<=500 and row['Ach % HO']>200:
        AchHO = 200
      elif row['Target']>500 and row['Target']<=1000 and row['Ach % HO']>400:
        AchHO = 400
      else:
        AchHO = row['Ach % HO']
        
        
    if row['Emp City']=='South Mumbai':
      if row['Target']<=80 and row['Ach % D1']>100:
        AchD1 = 100
      elif row['Target']>80 and row['Target']<=200  and row['Ach % D1']>200:
        AchD1 = 200
      elif row['Target']>200 and row['Target']<=400  and row['Ach % D1']>400:
        AchD1 = 400
      elif row['Target']>400 and row['Ach % D1']>400:
        AchD1 = row['Ach % D1']
      else:
        AchD1 = row['Ach % D1']
    elif row['Emp City']=='Kolkata':
      if row['Target']<=335 and row['Ach % D1']>100:
        AchD1 = 100
      elif row['Target']>=335 and row['Target']<=669 and row['Ach % D1']>400:
        AchD1 = 400
      else:
        AchD1 = row['Ach % D1']
    else:
      if row['Target']<=200 and row['Ach % D1']>100:
        AchD1 = 100
      elif row['Target']>200 and row['Target']<=500 and row['Ach % D1']>200:
        AchD1 = 200
      elif row['Target']>500 and row['Target']<=1000 and row['Ach % D1']>400:
        AchD1 = 400
      else:
        AchD1 = row['Ach % D1']

  else:
    AchHO = row['Ach % HO']
    AchD1 = row['Ach % D1']
		
  return AchHO,AchD1
			
EmpIPs[['Ach % HO','Ach % D1']] = EmpIPs.apply(payoutCapping, axis = 1 , result_type = 'expand')

############################################################################


import math
def custom_round(row):
  base = 20
  HO = 0
  D1 = 0
  if row['Ach % HO'] >= 0 and row['Closure Month'] == 9:
    HO =  int(base * (math.floor(row['Ach % HO']*1.2/base)))
  elif row['Ach % HO'] >= 0 and row['Closure Month'] != 9:
    HO =  int(base * (math.floor(row['Ach % HO']/base)))

  if row['Ach % D1'] >= 0 and row['Closure Month'] == 9:
    D1 = int(base * (math.floor(row['Ach % D1']*1.2/base)))
  elif row['Ach % D1'] >= 0 and row['Closure Month'] != 9:
    D1 = int(base * (math.floor(row['Ach % D1']/base)))  
  return HO,D1


EmpIPs[['Ach % HO binned','Ach % D1 binned']] = EmpIPs.apply(custom_round, axis = 1, result_type ='expand')

############################################################################

EmpIPs = pd.merge(EmpIPs,TSMPayoutSlabs, left_on = ['Ach % HO binned','Closure Month'], right_on = ['% Target Achieved','Closure Month'], how ='left')
EmpIPs.rename({'Incentive %':'Incentive % (HO)', 'Amount':'Amount (HO)','% Target Achieved':'% Target Slab Achieved (HO)'}, inplace = True, axis = 1)
EmpIPs = pd.merge(EmpIPs,TSMPayoutSlabs, left_on = ['Ach % D1 binned','Closure Month'], right_on = ['% Target Achieved','Closure Month'], how ='left')
EmpIPs.rename({'Incentive %':'Incentive % (D1)', 'Amount':'Amount (D1)','% Target Achieved':'% Target Slab Achieved (D1)'}, inplace = True, axis = 1)



############################################################################

# variableCTC = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Variable Payout/October 2021/TSM ASM - Variable Payout.csv")
# EmpIPs = pd.merge(EmpIPs,variableCTC[['Mail-ID','Monthly Variable']], left_on = ['Emp Email ID'], right_on = ['Mail-ID'], how = 'left')

# EmpIPs.loc[~EmpIPs['Monthly Variable'].isna(), 'Amount (HO)'] = EmpIPs['Amount (HO)']*EmpIPs['Monthly Variable']/15000
# EmpIPs.loc[~EmpIPs['Monthly Variable'].isna(), 'Amount (D1)'] = EmpIPs['Amount (D1)']*EmpIPs['Monthly Variable']/15000

# ############################################################################

# EmpIPs.loc[EmpIPs['Emp City'].isin({'Gurgaon','Ahmedabad','Pune','Jaipur','Navi Mumbai','Nagpur','Kolkata','Ahmedabad','Jaipur','Nagpur','Kochi'}), 'Sales Leader'] = 'ayanava.dasgupta@nobroker.in'
# EmpIPs.loc[EmpIPs['Emp City'].isin({'Mumbai','South Mumbai','Noida'}), 'Sales Leader'] = 'shobhna.arora@nobroker.in'
# EmpIPs.loc[EmpIPs['Emp City'].isin({'Hyderabad','Bangalore','Chennai'}), 'Sales Leader'] = 'paily.chiranjeevi@nobroker.in'


# EmpIPs = EmpIPs[['Emp Email ID', 'Emp City', 'Role', 'Team',
#        'Reporting supervisor Mail ID', 'Emp Status', 'Tenure','Source',
#        'In PIP/ On Notice Status','Closure Month', 'Target', 'Sales Leader',
#        'IP (HO)', 'IP (HO) - Team','Ach % HO', 'Incentive % (HO)','Amount (HO)',
#        'IP (D1)', 'Ach % D1', 'Incentive % (D1)','Amount (D1)','Monthly Variable']]       

# EmpIPs.drop_duplicates(inplace = True)


In [ ]:
variableCTC = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/TillAugRawMasters/Sales Targets/Variable Payout/January 2022/Variable Payout.csv")
variableCTC[['Effective Date']] = variableCTC[['Effective Date']].apply(changeDateFormat)
variableCTC.loc[:, 'Effective Month'] = pd.to_datetime(variableCTC['Effective Date']).dt.month
# EmpIPs1 = pd.merge(EmpIPs,variableCTC[['Mail-ID','Effective Month','Monthly Variable']], left_on = ['Emp Email ID','Closure Month'], right_on = ['Mail-ID', 'Effective Month'], how = 'left')
EmpIPs1 = pd.merge(EmpIPs,variableCTC[['Mail-ID','Effective Month','Monthly Variable']], left_on = ['Emp Email ID'], right_on = ['Mail-ID'], how = 'left')


EmpIPs1.loc[(~EmpIPs1['Monthly Variable'].isna()) & (EmpIPs1['Closure Month'] >= EmpIPs1['Effective Month']), 'Amount (HO)'] = EmpIPs1['Amount (HO)']*EmpIPs1['Monthly Variable']/15000
EmpIPs1.loc[(~EmpIPs1['Monthly Variable'].isna()) & (EmpIPs1['Closure Month'] >= EmpIPs1['Effective Month']), 'Amount (D1)'] = EmpIPs1['Amount (D1)']*EmpIPs1['Monthly Variable']/15000


In [ ]:
variableCTC[variableCTC['Mail-ID']=='pretti.prahladan@nobroker.in']

,Unnamed: 0,EMP-ID,Mail-ID,Variable Payout,Monthly Variable,Effective Date,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Effective Month
121,121,NBTS6576,pretti.prahladan@nobroker.in,202500,16875.0,2021-01-08,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
closures[closures['Closure Done By (Primary Email)']=='ezaz.rashid@nobroker.in']


,Closure Type,Closure Date,Original Closure Number,Closure Number,Closure Form Filled Date,Closure Form Filled Time,Closure Month,Closure Form Filled By,Team Type,Lead Type,...,RDDownloadMultiplier,D1 Product,VMSProductMultiplierD1,ERPProductMultiplierD1,NetMultiplier VMS D1,NetMultiplier ERP D1,NetMultiplier D1,IPs D1,PrimaryIPsD1,SecondaryIPsD1
815,New Closure,2021-10-30,1019004.0,1019004,2021-10-31,2021-10-31 21:55:08,10,yogesh.darra@nobroker.in,B2C Team,ASP,...,0.0,VMS only,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
823,New Closure,2021-10-30,1019397.0,1019397,2021-10-31,2021-10-31 22:32:34,10,yogesh.darra@nobroker.in,B2C Team,ASP,...,1.0,ERP only,0.0,1.0,0.0,1.0,1.0,30.0,15.0,15.0
1419,Upsell (VMS/ERP),2021-12-31,1019397.0,1020540,2021-12-31,2021-12-31 20:15:45,12,yogesh.darra@nobroker.in,B2C Team,City,...,1.0,VMS only,1.0,0.0,1.0,0.0,1.0,25.0,25.0,0.0


In [ ]:
EmpIPs[EmpIPs['Emp Email ID']=='pretti.prahladan@nobroker.in']


,Emp Email ID,Emp City,Role,Team,Reporting supervisor Mail ID,Emp Status,Tenure,Source,In PIP/ On Notice Status,Closure Month,...,Ach % HO,Ach % D1,Ach % HO binned,Ach % D1 binned,% Target Slab Achieved (HO),Incentive % (HO),Amount (HO),% Target Slab Achieved (D1),Incentive % (D1),Amount (D1)
820,pretti.prahladan@nobroker.in,Gurgaon,TSM,Mixed,nischaya.gupta@nobroker.in,Inactive,755.0,Internal Sourcing,,1.0,...,53.333333,60.000000,40,60,40.0,50.0,7500.0,60.0,90.0,13500.0
821,pretti.prahladan@nobroker.in,Gurgaon,TSM,NaN,NaN,Inactive,755.0,Internal Sourcing,,2.0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
822,pretti.prahladan@nobroker.in,Gurgaon,TSM,NaN,NaN,Inactive,755.0,Internal Sourcing,,3.0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
823,pretti.prahladan@nobroker.in,Gurgaon,TSM,NaN,NaN,Inactive,755.0,Internal Sourcing,,8.0,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN
824,pretti.prahladan@nobroker.in,Gurgaon,TSM,Sales,nischaya.gupta@nobroker.in,Inactive,755.0,Internal Sourcing,,9.0,...,62.513235,59.522059,60,60,60.0,90.0,13500.0,60.0,90.0,13500.0
825,pretti.prahladan@nobroker.in,Gurgaon,TSM,Sales,nischaya.gupta@nobroker.in,Inactive,755.0,Internal Sourcing,,10.0,...,46.850000,46.850000,40,40,40.0,50.0,7500.0,40.0,50.0,7500.0
1477,pretti.prahladan@nobroker.in,Gurgaon,TSM,Central,nischaya.gupta@nobroker.in,Inactive,755.0,Internal Sourcing,,11.0,...,5.583333,5.583333,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1478,pretti.prahladan@nobroker.in,Gurgaon,TSM,Mixed,nischaya.gupta@nobroker.in,Inactive,755.0,Internal Sourcing,,12.0,...,35.944444,35.944444,20,20,20.0,20.0,3000.0,20.0,20.0,3000.0


In [ ]:
EmpIPs.shape

(1545, 24)

#Uniview Merge

In [ ]:
dump_name = 'MIS-Tab1-Prospect-A new 16 dec'
Lead = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
Lead = Lead[['lead_id', 'employee_code','created_on', 'close_date','source', 'tag']]

Lead = Lead[~(Lead['employee_code'].isna())]
Lead[['created_on', 'close_date']] = Lead[['created_on', 'close_date']].apply(changeDateFormat)
Lead[['employee_code']] = Lead[['employee_code']].apply(cleanText)

Lead.rename({'lead_id':'Lead ID', 'employee_code':'Lead Created By','created_on':'Lead Creation Date', 'close_date':'Lead Closure Date',
       'source':'Lead Source', 'tag':'Lead Tag'}, inplace= True, axis = 1)

###############################################

dump_name = 'MIS-Tab3-Closure'
ClosuresUniview = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
ClosuresUniview = ClosuresUniview[['lead_id','closure_number','closure_date']]
ClosuresUniview[['closure_date']] = ClosuresUniview[['closure_date']].apply(changeDateFormat)
ClosuresUniview.rename({'lead_id':'Lead ID','closure_number':'Closure Number','closure_date':'Closure Date (Uniview)'}, inplace = True, axis = 1)

Lead = pd.merge(Lead, HR[['Emp Code','Emp Email ID','Role']], left_on = 'Lead Created By', right_on = 'Emp Code', how = 'left')
Lead = pd.merge(Lead, ClosuresUniview, on = 'Lead ID', how = 'right')

###############################################

dump_name = 'MIS-Tab2-Meeting-A with meeting start date filter'
meeting = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
#meeting = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/MIS-Tab2-Meeting-A with meeting start date filter -AVI.csv",parse_dates=True)

meeting = meeting[meeting['status'] == 'completed']
meeting = meeting[meeting['meeting_type'].isin({'mc_meeting','followup','sm_meeting','builder_meeting'})]

meeting = meeting[['Lead_id','meeting_id','start_date','end_date']]
meeting.rename({'Lead_id':'Lead ID','meeting_id':'Meeting ID', 'start_date':'Meeting Start Date','end_date':'Meeting End Date'}, inplace =True, axis = 1)
meeting[['Meeting Start Date','Meeting End Date']] = meeting[['Meeting Start Date','Meeting End Date']].apply(changeDateTimeFormat)

meeting['Meeting Start Time'] = meeting['Meeting Start Date'].dt.time
meeting['Total Meeting Duration (mins)'] = meeting['Meeting End Date'] - meeting['Meeting Start Date']
meeting['Total Meeting Duration (mins)'] = (meeting['Total Meeting Duration (mins)']/pd.Timedelta(minutes=1)).astype(int)


###
meeting = meeting[~(meeting['Total Meeting Duration (mins)'].isna())]
meeting = meeting[(meeting['Total Meeting Duration (mins)'] >= 10) & (meeting['Total Meeting Duration (mins)'] <= 240)] 
###


meeting[['Meeting Start Date','Meeting End Date']] = meeting[['Meeting Start Date','Meeting End Date']].apply(changeDateFormat)
#meeting = meeting[meeting['Meeting Start Date'] == meeting['Meeting End Date']]
meeting.drop(['Meeting End Date'], inplace = True, axis = 1)
###


meetingSummary = meeting.groupby('Lead ID').agg({'Meeting ID':'count','Total Meeting Duration (mins)':'sum'}).reset_index()
meetingSummary.rename({'Meeting ID':'Completed Meeting Count'}, inplace = True, axis = 1)


###############################################

dump_name = 'Questionnaire responses'
DemoForm = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
#DemoForm = pd.read_csv("/content/drive/MyDrive/Insights_Datalake/Demos.csv",parse_dates=True)

DemoForm.rename({'Form added date':'Form Filled Date',  'form_added_by':'Form Filled By',
                 'meeting__lead_id':'Lead ID', 'meeting__id':'Meeting ID', 'meeting__created_on':'Meeting Creation Date','meeting__end_date':'Meeting End Date', 'meeting__start_date':'Meeting Start Date', 
                 'meeting__meeting_title':'Meeting Title','meeting__meeting_type':'Meeting Client Type',
                 'meeting__product_demo_given':'Product Demo Given', 'meeting__status':'Meeting Status'}, inplace = True, axis = 1)

DemoForm[['Form Filled Date','Meeting Creation Date','Meeting Start Date','Meeting End Date']] = DemoForm[['Form Filled Date','Meeting Creation Date','Meeting Start Date','Meeting End Date']].apply(changeDateFormat)

#DemoForm = DemoForm[(DemoForm['Form Filled Date'] == DemoForm['Meeting Start Date']) & (DemoForm['Meeting End Date'] == DemoForm['Meeting Start Date'])]
DemoForm = DemoForm[DemoForm['Form template name'] == 'Demo form']

DemoForm = DemoForm[['Lead ID','Meeting ID', 'Meeting Status', 'Meeting Client Type',
                     'Meeting Creation Date','Meeting Start Date', 'Meeting End Date', 'Product Demo Given','mandatory',
                     'Form Filled Date', 'Form Filled By',
                     'question_label','question_data_type', 'text_value_response','numeric_value_response', 'date_value_response', 'bool_value_response']]

DemoForm.loc[DemoForm['question_label'] == 'Enter POC phone no.', 'numeric_value_response'] = -1
DemoForm['date_value_response'] = pd.to_numeric(DemoForm[(~DemoForm['date_value_response'].isna())]['date_value_response']).apply(lambda x: datetime.fromtimestamp(x/1000))
DemoForm[['date_value_response']] = DemoForm[['date_value_response']].apply(changeDateFormat)

###############################################

# dump_name = 'MIS-Tab6-Demo-Sheet with date filter'
# DemoRec = pd.read_csv(file_path + DumpDate + "/Deepa Raghav/"+ dump_name +".csv",parse_dates=True)
# DemoRec = DemoRec[['lead_id', 'meeting_id','call_recording']]
# DemoRec.rename({'lead_id':'Lead ID','meeting_id':'Meeting ID','call_recording':'Recording Link'}, inplace = True, axis = 1)

dump_name = 'Demo Recording (>15mins)'
DemoRec = pd.read_csv(file_path + DumpDate + "/Akash/"+ dump_name +".csv",parse_dates=True)
DemoRec = DemoRec[['lead_id', 'meeting_id','download_url']]
DemoRec.rename({'lead_id':'Lead ID','meeting_id':'Meeting ID','download_url':'Recording Link'}, inplace = True, axis = 1)

# DemoRec = DemoRec[['meeting_id','download_url']]
# DemoRec.rename({'meeting_id':'Meeting ID','download_url':'Recording Link'}, inplace = True, axis = 1)

####################################################

masterDemoRaw = pd.merge(DemoForm,DemoRec, left_on = ['Lead ID','Meeting ID'], right_on = ['Lead ID','Meeting ID'], how = 'left')
# masterDemoRaw = pd.merge(DemoForm,DemoRec, left_on = ['Meeting ID'], right_on = ['Meeting ID'], how = 'left')

masterDemoRaw = pd.merge(masterDemoRaw,HR, left_on = ['Form Filled By'], right_on = ['Emp Email ID'], how = 'left')

masterDemoRaw = masterDemoRaw[['Form Filled Date', 'Form Filled By', 'Lead ID', 'Meeting ID', 
                               'Meeting Status', 'Meeting Client Type','Meeting Creation Date', 'Meeting Start Date', 'Meeting End Date', 'Product Demo Given', 'mandatory', 
                         'question_label', 'question_data_type', 'text_value_response','numeric_value_response', 'date_value_response', 'bool_value_response','Recording Link']]

masterDemo = masterDemoRaw.copy()


masterDemo = masterDemo[masterDemo['question_label'].isin({ 'Demo Participants',
              'Enter demo duration ( in minutes)', 'Select mode of demo',
       'Type of Demo', 'Select POC Designation',
       'No. of people present during demo <society end>',
       'Select demo date',      
       'Email ID of other TSM/ ASM', 'Span of Demo',
       "Competitor active for (Note: Please ensure correct marking for multipliers. Any closures without declaration of competitor during demo won't be given multiplier)",
       'Competitor active in the society at time of demo'})]
masterDemo.loc[masterDemo['question_data_type'].str.contains('TEXT'), 'question_data_type'] = 'TEXT'
masterDemo.loc[masterDemo['question_data_type'].str.contains('DATE'), 'question_data_type'] = 'DATE'
masterDemo.loc[masterDemo['question_data_type'].str.contains('NUMBER'), 'question_data_type'] = 'NUMERIC'

def fetchData(row):
  if row['question_data_type'] == 'TEXT':
    return row['text_value_response']
  elif row['question_data_type'] == 'DATE':
    return row['date_value_response']
  elif row['question_data_type'] == 'NUMERIC':
    return row['numeric_value_response']


masterDemo['Demo Date']=masterDemo[masterDemo['question_label']=='Select demo date'].apply(fetchData,axis=1)
masterDemo[['Demo Date']] = masterDemo[['Demo Date']].apply(changeDateFormat)
masterDemo['Demo Date'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Date'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Type']=masterDemo[masterDemo['question_label']=='Type of Demo'].apply(fetchData,axis=1)
masterDemo['Demo Type'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Type'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Society POC']=masterDemo[masterDemo['question_label']=='Select POC Designation'].apply(fetchData,axis=1)
masterDemo['Demo Society POC'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Society POC'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Participants']=masterDemo[masterDemo['question_label']=='Demo Participants'].apply(fetchData,axis=1)
masterDemo.loc[masterDemo['Demo Participants'].isin({',SM','SM'}), 'Demo Participants'] = 'SM'
masterDemo.loc[masterDemo['Demo Participants'].isin({',MC','MC'}), 'Demo Participants'] = 'MC'
masterDemo.loc[masterDemo['Demo Participants'].isin({'SM,MC', 'MC,SM', ',SM,MC',',MC,SM'}), 'Demo Participants'] = 'MC + SM'
masterDemo['Demo Participants'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Participants'].apply(lambda x: x.ffill().bfill())


masterDemo['Demo Duration (in mins)']=masterDemo[masterDemo['question_label']=='Enter demo duration ( in minutes)'].apply(fetchData,axis=1)
masterDemo['Demo Duration (in mins)'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Duration (in mins)'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Mode']=masterDemo[masterDemo['question_label']=='Select mode of demo'].apply(fetchData,axis=1)
masterDemo['Demo Mode'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Mode'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Participant Count']=masterDemo[masterDemo['question_label']=='No. of people present during demo <society end>'].apply(fetchData,axis=1)
masterDemo['Demo Participant Count'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Participant Count'].apply(lambda x: x.ffill().bfill())

masterDemo['Demo Team Type']=masterDemo[masterDemo['question_label']=='Span of Demo'].apply(fetchData,axis=1)
masterDemo['Demo Team Type'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Team Type'].apply(lambda x: x.ffill().bfill())

# masterDemo['Demo Team Members']=masterDemo[masterDemo['question_label']=='Email ID of other TSM/ ASM'].apply(fetchData,axis=1)
# masterDemo['Demo Team Members'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Demo Team Members'].apply(lambda x: x.ffill().bfill())

masterDemo['Competitor Name']=masterDemo[masterDemo['question_label']=='Competitor active in the society at time of demo'].apply(fetchData,axis=1)
masterDemo['Competitor Name'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Competitor Name'].apply(lambda x: x.ffill().bfill())

masterDemo['Competitor Product']=masterDemo[masterDemo['question_label']=='Competitor active for (Note: Please ensure correct marking for multipliers. Any closures without declaration of competitor during demo won\'t be given multiplier)'].apply(fetchData,axis=1)
masterDemo['Competitor Product'] = masterDemo.groupby(['Lead ID', 'Meeting ID','Form Filled By','Form Filled Date'], sort=False)['Competitor Product'].apply(lambda x: x.ffill().bfill())


masterDemo.drop(['mandatory','question_label','question_data_type', 'text_value_response', 'numeric_value_response',
       'date_value_response', 'bool_value_response'], axis = 1, inplace=True)

masterDemo = masterDemo.drop_duplicates()

masterDemo = masterDemo[['Form Filled Date', 'Form Filled By', 'Meeting ID', 'Demo Date', 'Demo Type','Demo Mode', 'Demo Duration (in mins)', 'Recording Link']]


In [ ]:
meeting1 = pd.merge(meeting, masterDemo, on = 'Meeting ID', how = 'left')

meeting1 = meeting1[~meeting1['Recording Link'].isna()]
meeting1 = meeting1.sort_values(['Demo Duration (in mins)']).groupby(['Lead ID']).tail(1)


meeting2 = pd.merge(meeting, masterDemo, on = 'Meeting ID', how = 'left')

meetingSummary = meeting2.groupby('Lead ID').agg({'Meeting ID':'count','Total Meeting Duration (mins)':'mean','Form Filled Date':'count','Demo Duration (in mins)':'mean'}).reset_index()
meetingSummary.rename({'Meeting ID':'Completed Meeting Count','Form Filled Date':'Demos Given'}, inplace = True, axis = 1)


meetingSummary = pd.merge(meetingSummary,meeting1[['Lead ID','Form Filled By','Demo Date','Recording Link']], on = 'Lead ID', how = 'left')

leadMaster = pd.merge(Lead,meetingSummary, on = 'Lead ID', how ='left')
leadMaster = leadMaster[leadMaster['Lead Closure Date'] >= pd.to_datetime("2021-08-01").date()]

In [ ]:
closures = pd.merge(leadMaster,closures, on = 'Closure Number', how = 'right')

In [ ]:
closures.shape

(5584, 160)

#Backup and Writing

In [ ]:
closures.shape

(5584, 160)

In [ ]:
# closures.loc[:,'ClosureActivity'] = ""

closures = closures[['Lead ID', 'Lead Created By', 'Lead Creation Date', 'Lead Closure Date','Lead Source', 'Lead Tag', 'Emp Code', 'Emp Email ID', 'Role',
                     'Completed Meeting Count','Total Meeting Duration (mins)', 'Demos Given', 'Demo Duration (in mins)', 'Form Filled By', 'Demo Date','Recording Link',                      
                     'Closure Type','Closure Month','Closure Date', 'Closure Date (Uniview)','Original Closure Number','Closure Number','Closure Form Filled Date','Closure Form Filled By',
                     'Closure Done By (Primary Email)','Primary BD City','Primary BD Role', 'Closure Done By (Secondary Email)','Secondary BD City','Secondary BD Role',
                     'Team Type','Lead Type', 'Society Name (Sales)', 'Society Type','Property Type', 
                     'Closure City', 'VMSTrialPeriod', 'ERPTrialPeriod',
                     'Total Flats','Actual Flats','Sold Flats','VMS Sold Flats',
                     'Closure Form Filling TAT','WrongPrimaryBDMail','WrongSecondaryBDMail','WrongHOProduct','CrossCityClosure','DuplicateClosureNumber','WrongOriginalClosureNumber','ClosureActivity',
                     'VMSHOStatus','ERPHOStatus','VMS Sold Flats Mismatch','ERP Sold Flats Mismatch','VMSHOClosureTAT','ERPHOClosureTAT','VMSHOClosureFilledTAT','ERPHOClosureFilledTAT',
                     'VMSHOFilledDate','VMSHOFilledEmail','VMSHOMeetingID','VMSHODate','Occupied Flats (VMS)','Sold Flats (VMS)',
                     'ERPHOFilledDate','ERPHOFilledEmail','ERPHOMeetingID','ERPHODate','Occupied Flats (ERP)','Sold Flats (ERP)',
                     'City Type','FocusLead Multiplier', 'CrossCityFactor', 'Premium Multiplier',
                     'VMSProductMultiplier','VMSTrialReductionFactorD4','VMSTrialReductionFactorPIA', 'Competition Multiplier (Claimed) - VMS','Competition Multiplier (PIA) - VMS',
                     'ERPProductMultiplier','BillMigrationMultiplier', 'Competition Multiplier (Claimed) - ERP','Competition Multiplier (PIA) - ERP',                     
                     'Sales Product','HO Product','D1 Product','NetMultiplier D4','NetMultiplier HO','RDDownloadMultiplier','NetMultiplier D1',
                     'IPs D4', 'IPs HO', 'IPs D1', 'PrimaryIPsHO','SecondaryIPsHO', 'PrimaryIPsD1','SecondaryIPsD1',
                     'Possible Kibana Name Match','Auto-Matched Kibana ID','Kibana ID','Kibana Creation Date','RD Perc','Download Perc',
                     'combined IPR','Society AccountStatus','Bank Addition Date', 'Posted Date','Deletion Status','Bill Types','Flat Count','Total Posted Amount in K',
                     'Payment Unique Flats','Total Payment Amount in K','Vendor Bill Added (Last 30 Days)','Total Receipt Entries ',
                     'First Entry Date','Avg Total Entry(7 Days)','Avg Non-Staff Entry(7 Days)','RA Perc',
                     'Closure Proof Link','Premimum Factor Proof Link','Competition Replaced','Competition Proof Link (VMS)', 
                     'Competition Proof Link (ERP)', 'VMS Competitor being replaced','ERP Competitor being replaced',
                     'Remarks on MC (if any)','MigrationMCConfirmation', 'Liked Features Form Filled', 'HO Meeting ID', 'ASP Name',
                     'monthDays','year','VMSFlatsNew','ERPFlatsNew'
]]

closures.drop_duplicates(inplace=True)

In [ ]:
sheet  = gc.open_by_url('https://docs.google.com/spreadsheets/d/1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk/edit#gid=1583239167')

sheet =  sheet.get_worksheet(0)
closuresbackup = pd.DataFrame(sheet.get_all_records())

In [ ]:
closures["Last Updated Datetime"] = pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%d-%B-%Y - %H:%M")

In [ ]:
pd.to_datetime(datetime.now()).tz_localize('UTC').tz_convert('Asia/Kolkata').strftime("%d-%B-%Y - %H:%M")

'24-August-2022 - 13:44'

In [ ]:
spreadSheetId = "1O6XP_LqxlymjASE-0mM2nxju4bT2avUTVwyeqnX_MFk"
#rangeName = "ClosureHOMaster"
rangeName = "LeadMaster"

wb = gc.open_by_key(spreadSheetId)
ws = wb.worksheet(rangeName)
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, closures.sort_values(by=['Closure Date'], ascending = False))

# rangeName = "EmpIPMonth"
rangeName = "EmpIP"
ws = wb.worksheet(rangeName)
ws.clear()

# APPEND DATA TO SHEET
set_with_dataframe(ws, EmpIPs.sort_values('Amount (HO)', ascending= False))